In [1]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv("./Car_Dheko_datasets/Car_Dheko.csv", low_memory=False)

#maximize the columns
pd.set_option("display.max_columns",None)

In [2]:
df.shape

(8369, 325)

In [3]:
df.reset_index()

,index,new_car_detail_it,new_car_detail_ft,new_car_detail_bt,new_car_detail_km,new_car_detail_transmission,new_car_detail_ownerNo,new_car_detail_owner,new_car_detail_oem,new_car_detail_model,new_car_detail_modelYear,new_car_detail_centralVariantId,new_car_detail_variantName,new_car_detail_price,new_car_detail_priceActual,new_car_detail_priceSaving,new_car_detail_priceFixedText,new_car_detail_trendingText_imgUrl,new_car_detail_trendingText_heading,new_car_detail_trendingText_desc,new_car_overview_heading,new_car_overview_top_0_key,new_car_overview_top_0_value,new_car_overview_top_0_icon,new_car_overview_top_1_key,new_car_overview_top_1_value,new_car_overview_top_1_icon,new_car_overview_top_2_key,new_car_overview_top_2_value,new_car_overview_top_2_icon,new_car_overview_top_3_key,new_car_overview_top_3_value,new_car_overview_top_3_icon,new_car_overview_top_4_key,new_car_overview_top_4_value,new_car_overview_top_4_icon,new_car_overview_top_5_key,new_car_overview_top_5_value,new_car_overview_top_5_icon,new_car_overview_top_6_key,new_car_overview_top_6_value,new_car_overview_top_6_icon,new_car_overview_top_7_key,new_car_overview_top_7_value,new_car_overview_top_7_icon,new_car_overview_top_8_key,new_car_overview_top_8_value,new_car_overview_top_8_icon,new_car_overview_top_9_key,new_car_overview_top_9_value,new_car_overview_top_9_icon,new_car_overview_bottomData,new_car_feature_heading,new_car_feature_top_0_value,new_car_feature_top_1_value,new_car_feature_top_2_value,new_car_feature_top_3_value,new_car_feature_top_4_value,new_car_feature_top_5_value,new_car_feature_top_6_value,new_car_feature_top_7_value,new_car_feature_data_0_heading,new_car_feature_data_0_subHeading,new_car_feature_data_0_list_0_value,new_car_feature_data_0_list_1_value,new_car_feature_data_0_list_2_value,new_car_feature_data_0_list_3_value,new_car_feature_data_0_list_4_value,new_car_feature_data_0_list_5_value,new_car_feature_data_0_list_6_value,new_car_feature_data_0_list_7_value,new_car_feature_data_0_list_8_value,new_car_feature_data_0_list_9_value,new_car_feature_data_1_heading,new_car_feature_data_1_subHeading,new_car_feature_data_1_list_0_value,new_car_feature_data_1_list_1_value,new_car_feature_data_1_list_2_value,new_car_feature_data_1_list_3_value,new_car_feature_data_1_list_4_value,new_car_feature_data_1_list_5_value,new_car_feature_data_1_list_6_value,new_car_feature_data_2_heading,new_car_feature_data_2_subHeading,new_car_feature_data_2_list_0_value,new_car_feature_data_2_list_1_value,new_car_feature_data_2_list_2_value,new_car_feature_data_2_list_3_value,new_car_feature_data_2_list_4_value,new_car_feature_data_3_heading,new_car_feature_data_3_subHeading,new_car_feature_data_3_list_0_value,new_car_feature_data_3_list_1_value,new_car_feature_data_3_list_2_value,new_car_feature_data_3_list_3_value,new_car_feature_data_3_list_4_value,new_car_feature_data_3_list_5_value,new_car_feature_data_3_list_6_value,new_car_feature_data_3_list_7_value,new_car_feature_data_3_list_8_value,new_car_feature_data_3_list_9_value,new_car_feature_data_3_list_10_value,new_car_feature_data_3_list_11_value,new_car_feature_data_3_list_12_value,new_car_feature_commonIcon,new_car_specs_heading,new_car_specs_top_0_key,new_car_specs_top_0_value,new_car_specs_top_1_key,new_car_specs_top_1_value,new_car_specs_top_2_key,new_car_specs_top_2_value,new_car_specs_top_3_key,new_car_specs_top_3_value,new_car_specs_top_4_key,new_car_specs_top_4_value,new_car_specs_data_0_heading,new_car_specs_data_0_subHeading,new_car_specs_data_0_list_0_key,new_car_specs_data_0_list_0_value,new_car_specs_data_0_list_1_key,new_car_specs_data_0_list_1_value,new_car_specs_data_0_list_2_key,new_car_specs_data_0_list_2_value,new_car_specs_data_0_list_3_key,new_car_specs_data_0_list_3_value,new_car_specs_data_0_list_4_key,new_car_specs_data_0_list_4_value,new_car_specs_data_0_list_5_key,new_car_specs_data_0_list_5_value,new_car_specs_data_0_list_6_key,new_car_specs_data_0_list_6_value,new_car_specs_data_0_list_7_ke

In [4]:
# Remove the column if unnesscary
df.drop(columns=['new_car_detail_owner','new_car_detail_trendingText_imgUrl',
                 'new_car_detail_trendingText_heading','new_car_detail_trendingText_desc',
                 'new_car_detail_priceActual','new_car_detail_priceSaving','new_car_detail_priceFixedText'],inplace=True)


# Clean the 'km' column by removing commas and converting to integers
df['new_car_detail_km'] = df['new_car_detail_km'].str.replace(',', '').astype(int)

# Lambda function to convert price to float, including "Lakh" and "Crore"
convert_price = lambda price: int(float(
    price.replace('₹', '').replace(',', '').replace('Lakh', '').strip()
) * 100000 if 'Lakh' in price else float(
    price.replace('₹', '').replace(',', '').replace('Crore', '').strip()
) * 10000000 if 'Crore' in price else float(
    price.replace('₹', '').replace(',', '').strip()
)
)
# Apply the lambda function to the 'price' column
df['new_car_detail_price'] = df['new_car_detail_price'].apply(lambda x: convert_price(x) if pd.notnull(x) else pd.NA)
df['new_car_detail_oem'] = df['new_car_detail_oem'].replace('Mahindra Ssangyong', 'Mahindra SsangYong')

prefixes = ['new_car_detail_']  # Add more prefixes if needed



for prefix in prefixes:
    df.columns = df.columns.str.replace(prefix, '')


In [5]:
#drop columns in'new_car_overview'
df.drop(columns=['new_car_overview_heading','new_car_overview_top_0_key',
                 'new_car_overview_top_0_icon','new_car_overview_top_0_value',#here new_car_overview_top_0_value remove because new_car_detail_modelYear are same
                 'new_car_overview_top_1_key','new_car_overview_top_1_icon',
                 'new_car_overview_top_2_key','new_car_overview_top_2_icon',
                 'new_car_overview_top_2_value','new_car_overview_top_3_key',#here new_car_overview_top_2_value remove because new_car detail_ft are same
                 'new_car_overview_top_3_icon','new_car_overview_top_4_icon',
                'new_car_overview_top_4_key','new_car_overview_top_4_value',#here new_car_overview_top_4_value remove because new_car detail_km are same
                 'new_car_overview_top_5_icon','new_car_overview_top_5_key',
                 'new_car_overview_top_6_icon','new_car_overview_top_6_key',
                 'new_car_overview_top_7_key','new_car_overview_top_6_value',#here new_car_overview_top_6_value remove because new_car detail_ownerNo are same
                 'new_car_overview_top_7_icon','new_car_overview_top_8_key',
                 'new_car_overview_top_8_icon','new_car_overview_top_8_value',#here new_car_overview_top_8_value remove because new_car detail_transmission are same
                 'new_car_overview_top_9_key','new_car_overview_top_9_value',#here new_car_overview_top_9_value remove beacuase new_car_detail_modelYear are same
                 'new_car_overview_top_9_icon','car_links','new_car_overview_bottomData'], inplace=True)

#rename columns in 'new_car_overiew'
df.rename(columns={
                   'new_car_overview_top_1_value': 'Insurance Validity',
                  'new_car_overview_top_3_value': 'Seats',
                  'new_car_overview_top_5_value': 'RTO',
                  'new_car_overview_top_7_value': 'Engine Displacement'}, inplace=True)


# insurance column occurs fuel type to replace null vales
Insurance = ['Petrol', 'Diesel', 'Electric']
df['Insurance Validity'] = df['Insurance Validity'].apply(
    lambda y: np.nan if str(y).strip().lower() in [i.lower() for i in Insurance] else y)
df['Insurance Validity'] = df['Insurance Validity'].replace({'Third Party insurance': 'Third Party', 'Zero Dep': '0'})


#separate the numerical values
df['Seats'] = df['Seats'].apply(
    lambda a: int(a.replace('Seats', '').strip()) 
    if a.replace('Seats', '').strip().isdigit() and int(a.replace('Seats', '').strip()) < 10 
    else pd.NA)


Owner = ['First Owner','Second Owner','Third Owner','Fourth Owner', 'Fifth Owner','1995 cc','1061 cc','993 cc',
         '1197 cc','1799 cc','1999 cc','2446 cc','2993 cc','796 cc','796 cc','993 cc','998 cc','999 cc','Automatic','Manual']

# Convert the items in the Owner list to lowercase for case-insensitive comparison
Owner_lower = [item.lower() for item in Owner]

df['RTO'] = df['RTO'].apply(
    lambda b: pd.NA if str(b).strip().lower() in Owner_lower else b)

df['RTO'] = df['RTO'].replace({'HR26': "HP26", 'HR98': "HP98", 'HR01': "HP01", 'VK28': np.nan, 'TN 0': np.nan, 'HR70': "HP70", 'UK07': "UP07", 'HR08': "HP08", 'HR13': "HP13", 
'HR36': "HP36", 'HR51': "HP51",'TT07': "TS07", 'BR3F': np.nan, 'HR12': "HP12", 'TN 1': "TN01", 'Up15': "UP15", 'HR72': "HP72", 'PH65': "PB65", 
'DD4L': "DL4L", 'HR89': "HP89", 'HR29': "HP29", 'UK14': "UP14", 'HR06': "HP06", 'HR87': "HP87", 'HR05': "HP05", 'Hr22': "HP22", 'HR16': "HP16", 
'DD8L': 'DL8L', 'HR90': "HP90", 'HR52': "HP52", 'HR30': "HP30", 'HR19': "HP19", 'UK17': "UP17", 'HR03': "HP03", 'HR81': "HP81", 'Dl12': "DL12",
'UK08': "UP08", 'UK15': "UP15", 'DD06': "DL06", 'HR10': "HP10", 'UK06': "UP06", 'HR42': "HP42", 'HR55': "HP55", 'RH14': "RJ14",'TA13': "TS13", 
'KK04': "KA04", 'TA10': "TS10", 'TA07':"TS07", 'Wb10': "WB10", 'AA28': "AP28", 'Wb06': "WB06", 'Wb24': "WB24", 'Wb02': "WB02", 'TT02': "TS02", 
'Wb16': "WB16", 'TD07': "TS07", 'HE26': "HP26", 'RR23': "RJ23", 'Wb08': "WB08", 'wB24': "WB24", 'HR27': "HP27", 'R14C': np.nan, 'Wb34': "WB34",
'HR43': "HP43", 'TM02': "TS02", 'HR78': "HP78", 'HR35': "HP35", 'Wb14': "WB14", 'Wb26': "WB26", 'Wb12': "WB12", 'Wb52': "WB52", 'wb02': "WB02",
'wb52': "WB52", 'WW06': "WB06", 'wb24': "WB24", 'HR24': "HP24", 'HR14': "HP14", 'wb06': "WB06", 'UK18': "UP18"})


trans = ['Automatic', 'Manual', '2010', '2014', '2015', '2018', '2021']

# Convert the items in the trans list to lowercase for case-insensitive comparison
trans_lower = [m.lower() for m in trans]

# Apply the lambda function to process the 'Engine Displacement' column
df['Engine Displacement'] = df['Engine Displacement'].apply(
    lambda c: int(c.replace('cc', '').strip()) if 'cc' in str(c).lower() 
    else pd.NA if str(c).strip().lower() in trans_lower else c
)
# Replace specific values, set others to 0
df['Engine Displacement'] = df['Engine Displacement'].replace({1197: 1, 998: 2, 1199: 3, 1497: 4, 1248: 5})

# Set values not in the dictionary to 0
df['Engine Displacement'] = df['Engine Displacement'].where(df['Engine Displacement'].isin([1, 2, 3, 4, 5]), 0)

In [6]:
#drop columns in'new_car_feature'
df.drop(columns=['new_car_feature_commonIcon','new_car_feature_data_0_heading',
                 'new_car_feature_data_0_subHeading', 'new_car_feature_data_1_heading',
                 'new_car_feature_data_1_subHeading','new_car_feature_data_2_heading',  
                 'new_car_feature_data_2_subHeading', 'new_car_feature_data_3_heading',
                 'new_car_feature_data_3_subHeading','new_car_feature_heading',
                'new_car_feature_data_0_list_0_value', 'new_car_feature_data_0_list_34_value',
                'new_car_feature_data_0_list_35_value', 'new_car_feature_data_0_list_36_value',
                'new_car_feature_data_1_list_20_value', 'new_car_feature_data_1_list_21_value',
                 'new_car_feature_data_1_list_17_value', 'new_car_feature_data_2_list_24_value',
                 'new_car_feature_data_2_list_25_value', 'new_car_feature_data_2_list_26_value',
                 'new_car_feature_data_3_list_37_value', 'new_car_feature_data_3_list_38_value',
                 'new_car_feature_data_3_list_39_value', 'new_car_feature_data_3_list_40_value',
                 'new_car_feature_data_3_list_41_value', 'new_car_feature_data_3_list_42_value', 
                 'new_car_feature_data_3_list_43_value', 'new_car_feature_data_3_list_44_value',
                 'new_car_feature_data_3_list_46_value', 'new_car_feature_data_3_list_47_value',
                 'new_car_feature_data_4_list_14_value', 'new_car_feature_data_4_list_15_value',
                 'new_car_feature_data_4_list_16_value','new_car_feature_data_4_list_17_value',
                 'new_car_feature_data_4_list_18_value','new_car_feature_data_4_list_20_value'
                          
                ], inplace=True)

df.rename(columns={'new_car_feature_top_0_value': "Power Steering",
                  'new_car_feature_top_1_value': 'Power Windows Front',
                  'new_car_feature_top_2_value': 'Air Conditioner',
                   'new_car_feature_top_3_value': 'Heater',
                   'new_car_feature_top_4_value': 'Fog Lights Rear',
                   'new_car_feature_top_5_value': 'Fog Lights Front',
                   'new_car_feature_top_6_value': 'Anti Lock Braking System',
                   'new_car_feature_top_7_value': 'Central Locking'}, inplace=True)

#each column value_counts, which counts is high mention 1 otherwise 0

df['Power Steering'] = df['Power Steering'].apply(lambda k: 1 if isinstance(k, str) and k == "Power Steering" else 0)
df['Power Windows Front'] = df['Power Windows Front'].apply(lambda e: 1 if e == "Power Windows Front" else 0)
df['Air Conditioner'] = df['Air Conditioner'].apply(lambda f: 1 if f == "Air Conditioner" else 0)
df['Heater'] = df['Heater'].apply(lambda g: 1 if g == "Heater" else 0)
df['Fog Lights Rear'] = df['Fog Lights Rear'].apply(lambda h: 1 if h == "Fog Lights Rear" else 0)
df['Fog Lights Front'] = df['Fog Lights Front'].apply(lambda i: 1 if i == "Fog Lights Front" else 0)
df['Anti Lock Braking System'] = df['Anti Lock Braking System'].apply(lambda j: 1 if j == "Anti Lock Braking System" else 0)
df['Central Locking'] = df['Central Locking'].apply(lambda k: 1 if k == "Centeral Locking" else 0)


In [7]:
df.rename(columns={'new_car_feature_data_0_list_1_value':'Remote Fuel Lid Opener',
                   'new_car_feature_data_0_list_2_value': 'Power Windows Rear',
                   'new_car_feature_data_0_list_3_value': 'Air Quality Control',
                   'new_car_feature_data_0_list_4_value': 'Remote Trunk Opener',
                   'new_car_feature_data_0_list_5_value': 'Low Fuel Warning Light',
                   'new_car_feature_data_0_list_6_value': 'Accessory Power Outlet',
                   'new_car_feature_data_0_list_7_value': 'Vanity Mirror',
                   'new_car_feature_data_0_list_8_value': 'Trunk Light',
                   'new_car_feature_data_0_list_9_value': 'Rear Seat Headrest',
                   'new_car_feature_data_0_list_10_value': 'Rear Reading Lamp',
                   'new_car_feature_data_0_list_11_value': 'Rear Seat Centre Arm Rest',
                   'new_car_feature_data_0_list_12_value': 'Cup Holders Front',
                   'new_car_feature_data_0_list_13_value': 'Cup Holders Rear',
                   'new_car_feature_data_0_list_14_value': 'Multifunction Steering Wheel',
                   'new_car_feature_data_0_list_15_value': 'Rear ACVents',
                   'new_car_feature_data_0_list_16_value': 'Seat Lumbar Support',
                   'new_car_feature_data_0_list_17_value': 'Smart Access Card Entry',
                   'new_car_feature_data_0_list_18_value': 'Cruise Control',
                   'new_car_feature_data_0_list_19_value': 'Engine Start Stop Button',
                   'new_car_feature_data_0_list_20_value': 'Navigation System',
                   'new_car_feature_data_0_list_21_value': 'Gear Shift Indicator',
                   'new_car_feature_data_0_list_22_value': 'Voice Control',
                   'new_car_feature_data_0_list_23_value': 'Glove Box Cooling',
                   'new_car_feature_data_0_list_24_value': 'Tailgate Ajar',
                   'new_car_feature_data_0_list_25_value': 'Steering Wheel Gearshift Paddles',
                   'new_car_feature_data_0_list_26_value': 'Lane Change Indicator',
                   'new_car_feature_data_0_list_27_value': 'Luggage Hook And Net',
                   'new_car_feature_data_0_list_28_value' : 'Drive Modes',
                   'new_car_feature_data_0_list_29_value' : 'Remote Climate Control',
                   'new_car_feature_data_0_list_30_value' : 'Adjustable Headrest',
                   'new_car_feature_data_0_list_31_value' : 'Power Boot',
                   'new_car_feature_data_0_list_32_value' : 'Hands Free Tailgate',
                   'new_car_feature_data_0_list_33_value' : 'Smart Key Band'}, inplace=True)

#each column value_counts, which counts is high mention 1 otherwise 0
df['Remote Fuel Lid Opener'] = df['Remote Fuel Lid Opener'].apply(lambda k: 1 if isinstance(k, str) and k == "Remote Fuel Lid Opener" else 0)
df['Power Windows Rear'] = df['Power Windows Rear'].apply(lambda l: 1 if l == "Power Windows Rear" else 0)
df['Air Quality Control'] = df['Air Quality Control'].apply(lambda n: 1 if n == "Air Quality Control" else 0)
df['Remote Trunk Opener'] = df['Remote Trunk Opener'].apply(lambda o: 1 if o == "Remote Trunk Opener" else 0)
df['Low Fuel Warning Light'] = df['Low Fuel Warning Light'].apply(lambda p: 1 if p == "Low Fuel Warning Light" else 0)
df['Accessory Power Outlet'] = df['Accessory Power Outlet'].apply(lambda q: 1 if q == "Accessory Power Outlet" else 0)
df['Vanity Mirror'] = df['Vanity Mirror'].apply(lambda r: 1 if r == "Vanity Mirror" else 0)
df['Trunk Light'] = df['Trunk Light'].apply(lambda r: 1 if r == "Trunk Light" else 0)
df['Rear Seat Headrest'] = df['Rear Seat Headrest'].apply(lambda r: 1 if r == "Rear Seat Headrest" else 0)
df['Rear Reading Lamp'] = df['Rear Reading Lamp'].apply(lambda r: 1 if r == "Rear Reading Lamp" else 0)
df['Rear Seat Centre Arm Rest'] = df['Rear Seat Centre Arm Rest'].apply(lambda r: 1 if r == "Rear Seat Centre Arm Rest" else 0)
df['Cup Holders Front'] = df['Cup Holders Front'].apply(lambda r: 1 if r == "Cup Holders Front" else 0)
df['Cup Holders Rear'] = df['Cup Holders Rear'].apply(lambda r: 1 if r == "Cup Holders Rear" else 0)
df['Multifunction Steering Wheel'] = df['Multifunction Steering Wheel'].apply(lambda r: 1 if r == "Multifunction Steering Wheel" else 0)
df['Rear ACVents'] = df['Rear ACVents'].apply(lambda r: 1 if r == "Rear ACVents" else 0)
df['Seat Lumbar Support'] = df['Seat Lumbar Support'].apply(lambda r: 1 if r == "Seat Lumbar Support" else 0)
df['Smart Access Card Entry'] = df['Smart Access Card Entry'].apply(lambda r: 1 if r == "Smart Access Card Entry" else 0)
df['Cruise Control'] = df['Cruise Control'].apply(lambda r: 1 if r == "Cruise Control" else 0)
df['Engine Start Stop Button'] = df['Engine Start Stop Button'].apply(lambda r: 1 if r == "Engine Start Stop Button" else 0)
df['Navigation System'] = df['Navigation System'].apply(lambda r: 1 if r == "Navigation System" else 0)
df['Gear Shift Indicator'] = df['Gear Shift Indicator'].apply(lambda r: 1 if r == "Gear Shift Indicator" else 0)
df['Voice Control'] = df['Voice Control'].apply(lambda r: 1 if r == "Voice Control" else 0)
df['Glove Box Cooling'] = df['Glove Box Cooling'].apply(lambda r: 1 if r == "Glove Box Cooling" else 0)
df['Tailgate Ajar'] = df['Tailgate Ajar'].apply(lambda r: 1 if r == "Tailgate Ajar" else 0)
df['Steering Wheel Gearshift Paddles'] = df['Steering Wheel Gearshift Paddles'].apply(lambda r: 1 if r == "Steering Wheel Gearshift Paddles" else 0)
df['Lane Change Indicator'] = df['Lane Change Indicator'].apply(lambda r: 1 if r == "Lane Change Indicator" else 0)
df['Luggage Hook And Net'] = df['Luggage Hook And Net'].apply(lambda r: 1 if r == "Luggage Hook And Net" else 0)
df['Drive Modes'] = df['Drive Modes'].apply(lambda r: 1 if r == "Drive Modes" else 0)
df['Remote Climate Control'] = df['Remote Climate Control'].apply(lambda r: 1 if r == "Remote Climate Control" else 0)
df['Adjustable Headrest'] = df['Adjustable Headrest'].apply(lambda r: 1 if r == "Adjustable Headrest" else 0)
df['Power Boot'] = df['Power Boot'].apply(lambda r: 1 if r == "Power Boot" else 0)
df['Hands Free Tailgate'] = df['Hands Free Tailgate'].apply(lambda r: 1 if r == "Hands Free Tailgate" else 0)
df['Smart Key Band'] = df['Smart Key Band'].apply(lambda r: 1 if r == "Smart Key Band" else 0)

In [8]:
df.rename(columns={'new_car_feature_data_1_list_0_value': 'Leather Seats',
                   'new_car_feature_data_1_list_1_value': 'Leather Steering Wheel',
                   'new_car_feature_data_1_list_2_value': 'Adjustable Steering',
                   'new_car_feature_data_1_list_3_value': 'Digital Odometer',
                   'new_car_feature_data_1_list_4_value': 'Tachometer',
                   'new_car_feature_data_1_list_5_value': 'Electronic Multi Tripmeter',
                   'new_car_feature_data_1_list_6_value': 'Fabric Upholstery',
                   'new_car_feature_data_1_list_7_value': 'Glove Compartment',
                   'new_car_feature_data_1_list_8_value': 'Digital Clock',
                   'new_car_feature_data_1_list_9_value': 'Dual Tone Dashboard',
                   'new_car_feature_data_1_list_10_value': 'Outside Temperature Display',
                   'new_car_feature_data_1_list_11_value': 'Height Adjustable Driver Seat',
                   'new_car_feature_data_1_list_12_value': 'Driving Experience Control Eco',
                   'new_car_feature_data_1_list_13_value': 'Leather Wrap Gear Shift Selector',
                   'new_car_feature_data_1_list_14_value': 'Ventilated Seats',
                   'new_car_feature_data_1_list_15_value': 'Removable Convertible Top',
                   'new_car_feature_data_1_list_16_value': 'Roof Carrier',
                   'new_car_feature_data_1_list_18_value': 'Moon Roof',
                   'new_car_feature_data_1_list_19_value': 'Side Stepper'}, inplace=True)

#each column value_counts, which counts is high mention 1 otherwise 0
df['Leather Seats'] = df['Leather Seats'].apply(lambda r: 1 if r == "Leather Seats" else 0)
df['Leather Steering Wheel'] = df['Leather Steering Wheel'].apply(lambda r: 1 if r == "Leather Steering Wheel" else 0)
df['Adjustable Steering'] = df['Adjustable Steering'].apply(lambda s: 1 if s == "Adjustable Steering" else 0)
df['Digital Odometer'] = df['Digital Odometer'].apply(lambda t: 1 if t == "Digital Odometer" else 0)
df['Tachometer'] = df['Tachometer'].apply(lambda t: 1 if t == "Tachometer" else 0)
df['Electronic Multi Tripmeter'] = df['Electronic Multi Tripmeter'].apply(lambda u: 1 if u == "Electronic Multi Tripmeter" else 0)
df['Fabric Upholstery'] = df['Fabric Upholstery'].apply(lambda v: 1 if v == "Fabric Upholstery" else 0)
df['Glove Compartment'] = df['Glove Compartment'].apply(lambda r: 1 if r == "Glove Compartment" else 0)
df['Digital Clock'] = df['Digital Clock'].apply(lambda r: 1 if r == "Digital Clock" else 0)
df['Dual Tone Dashboard'] = df['Dual Tone Dashboard'].apply(lambda r: 1 if r == "Dual Tone Dashboard" else 0)
df['Outside Temperature Display'] = df['Outside Temperature Display'].apply(lambda r: 1 if r == "Outside Temperature Display" else 0)
df['Height Adjustable Driver Seat'] = df['Height Adjustable Driver Seat'].apply(lambda r: 1 if r == "Height Adjustable Driver Seat" else 0)
df['Driving Experience Control Eco'] = df['Driving Experience Control Eco'].apply(lambda r: 1 if r == " Driving Experience Control Eco" else 0)
df['Leather Wrap Gear Shift Selector'] = df['Leather Wrap Gear Shift Selector'].apply(lambda r: 1 if r == "Leather Wrap Gear Shift Selector" else 0)
df['Ventilated Seats'] = df['Ventilated Seats'].apply(lambda r: 1 if r == "Ventilated Seats" else 0)
df['Removable Convertible Top'] = df['Removable Convertible Top'].apply(lambda r: 1 if r == "Removable Convertible Top" else 0)
df['Roof Carrier'] = df['Roof Carrier'].apply(lambda r: 1 if r == "Roof Carrier" else 0)
df['Moon Roof'] = df['Moon Roof'].apply(lambda r: 1 if r == "Moon Roof" else 0)
df['Side Stepper'] = df['Side Stepper'].apply(lambda r: 1 if r == "Side Stepper" else 0)



In [9]:
df.rename(columns={'new_car_feature_data_2_list_0_value': 'Adjustable Head Lights',
                   'new_car_feature_data_2_list_1_value': 'Manually Adjustable Exterior Rear View Mirror',
                   'new_car_feature_data_2_list_2_value': 'Power Adjustable Exterior Rear View Mirror',
                   'new_car_feature_data_2_list_3_value': 'Electric Folding Rear View Mirror',
                   'new_car_feature_data_2_list_4_value': 'Rear Window Wiper',
                   'new_car_feature_data_2_list_5_value': 'Rear Window Washer',
                   'new_car_feature_data_2_list_6_value': 'Rear Window Defogger',
                   'new_car_feature_data_2_list_7_value': 'Alloy Wheels',
                   'new_car_feature_data_2_list_8_value': 'Integrated Antenna',
                   'new_car_feature_data_2_list_9_value': 'Rear Spoiler',
                   'new_car_feature_data_2_list_10_value': 'Sun Roof',
                   'new_car_feature_data_2_list_11_value': 'Outside Rear View Mirror Turn Indicators',
                   'new_car_feature_data_2_list_12_value': 'Chrome Grille',
                   'new_car_feature_data_2_list_13_value': 'Chrome Garnish',
                   'new_car_feature_data_2_list_14_value': 'Roof Rail',
                   'new_car_feature_data_2_list_15_value': 'Smoke Headlamps',
                   'new_car_feature_data_2_list_16_value': 'LEDDRLs',
                   'new_car_feature_data_2_list_17_value':'LEDHeadlights',
                   'new_car_feature_data_2_list_18_value':'LEDTaillights',
                   'new_car_feature_data_2_list_19_value':'LEDFog Lamps',
                   'new_car_feature_data_2_list_20_value':'Projector Headlamps',
                   'new_car_feature_data_2_list_21_value': 'Headlamp Washers',
                   'new_car_feature_data_2_list_22_value': 'Cornering Foglamps',
                   'new_car_feature_data_2_list_23_value': 'Cornering Headlamps'}, inplace=True)


df['Adjustable Head Lights'] = df['Adjustable Head Lights'].apply(lambda r: 1 if r == "Adjustable Head Lights" else 0)
df['Manually Adjustable Exterior Rear View Mirror'] = df['Manually Adjustable Exterior Rear View Mirror'].apply(lambda v: 1 if v == "Manually Adjustable Exterior Rear View Mirror" else 0)
df['Power Adjustable Exterior Rear View Mirror'] = df['Power Adjustable Exterior Rear View Mirror'].apply(lambda w: 1 if w == "Power Adjustable Exterior Rear View Mirror" else 0)
df['Electric Folding Rear View Mirror'] = df['Electric Folding Rear View Mirror'].apply(lambda z: 1 if z == "Electric Folding Rear View Mirror" else 0)
df['Rear Window Wiper'] = df['Rear Window Wiper'].apply(lambda aa: 1 if aa == "Rear Window Wiper" else 0)
df['Rear Window Washer'] = df['Rear Window Washer'].apply(lambda bp: 1 if bp == "Rear Window Washer" else 0)
df['Alloy Wheels'] = df['Alloy Wheels'].apply(lambda bq: 1 if bq == "Alloy Wheels" else 0)
df['Rear Window Defogger'] = df['Rear Window Defogger'].apply(lambda br: 1 if br == "Rear Window Defogger" else 0)
df['Integrated Antenna'] = df['Integrated Antenna'].apply(lambda bs: 1 if bs == "Integrated Antenna" else 0)
df['Rear Spoiler'] = df['Rear Spoiler'].apply(lambda bn: 1 if bn == "Rear Spoiler" else 0)
df['Sun Roof'] = df['Sun Roof'].apply(lambda r: 1 if r == "Sun Roof" else 0)
df['Outside Rear View Mirror Turn Indicators'] = df['Outside Rear View Mirror Turn Indicators'].apply(lambda r: 1 if r == "Outside Rear View Mirror Turn Indicators" else 0)
df['Chrome Grille'] = df['Chrome Grille'].apply(lambda r: 1 if r == "Chrome Grille" else 0)
df['Chrome Garnish'] = df['Chrome Garnish'].apply(lambda r: 1 if r == "Chrome Garnish" else 0)
df['Roof Rail'] = df['Roof Rail'].apply(lambda r: 1 if r == "Roof Rail" else 0)
df['Smoke Headlamps'] = df['Smoke Headlamps'].apply(lambda r: 1 if r == "Smoke Headlamps" else 0)
df['LEDDRLs'] = df['LEDDRLs'].apply(lambda r: 1 if r == "LEDDRLs" else 0)
df['LEDHeadlights'] = df['LEDHeadlights'].apply(lambda r: 1 if r == "LEDHeadlights" else 0)
df['LEDTaillights'] = df['LEDTaillights'].apply(lambda r: 1 if r == "LEDTaillights" else 0)
df['LEDFog Lamps'] = df['LEDFog Lamps'].apply(lambda r: 1 if r == "LEDFog Lamps" else 0)
df['Projector Headlamps'] = df['Projector Headlamps'].apply(lambda r: 1 if r == "Projector Headlamps" else 0)
df['Headlamp Washers'] = df['Headlamp Washers'].apply(lambda r: 1 if r == "Headlamp Washers" else 0)
df['Cornering Foglamps'] = df['Cornering Foglamps'].apply(lambda r: 1 if r == "Cornering Foglamps" else 0)
df['Cornering Headlamps'] = df['Cornering Headlamps'].apply(lambda r: 1 if r == "Cornering Headlamps" else 0)



In [10]:
df.rename(columns={'new_car_feature_data_3_list_0_value': 'Day Night Rear View Mirror',
                   'new_car_feature_data_3_list_1_value': 'Brake Assist',
                   'new_car_feature_data_3_list_2_value': 'Power Door Locks',
                   'new_car_feature_data_3_list_3_value': 'Child Safety Locks',
                   'new_car_feature_data_3_list_4_value': 'Anti Theft Alarm',
                   'new_car_feature_data_3_list_5_value': 'Driver Air Bag',
                   'new_car_feature_data_3_list_6_value': 'Passenger Air Bag',
                   'new_car_feature_data_3_list_7_value': 'Passenger Side Rear View Mirror',
                   'new_car_feature_data_3_list_8_value': 'Halogen Headlamps',
                   'new_car_feature_data_3_list_9_value': 'Rear Seat Belts',
                   'new_car_feature_data_3_list_10_value': 'Seat Belt Warning',
                   'new_car_feature_data_3_list_11_value': 'Door Ajar Warning',
                   'new_car_feature_data_3_list_12_value': 'Side Impact Beams',
                   'new_car_feature_data_3_list_13_value': 'Front Impact Beams',
                   'new_car_feature_data_3_list_14_value': 'Adjustable Seats',
                   'new_car_feature_data_3_list_15_value': 'Keyless Entry',
                   'new_car_feature_data_3_list_16_value': 'Centrally Mounted Fuel Tank',
                   'new_car_feature_data_3_list_17_value': 'Engine Immobilizer',
                   'new_car_feature_data_3_list_18_value': 'Engine Check Warning',
                   'new_car_feature_data_3_list_19_value': 'Crash Sensor',
                   'new_car_feature_data_3_list_20_value': 'Ebd',
                   'new_car_feature_data_3_list_21_value': 'Anti Theft Device',
                   'new_car_feature_data_3_list_22_value': 'Rear Camera',
                   'new_car_feature_data_3_list_23_value': 'Speed Sensing Auto Door Lock',
                   'new_car_feature_data_3_list_24_value': 'No Of Airbags',
                   'new_car_feature_data_3_list_25_value': 'Automatic Head Lamps',
                   'new_car_feature_data_3_list_26_value': 'Isofix Child Seat Mounts',
                   'new_car_feature_data_3_list_27_value': 'Pretensioners And Force Limiter Seatbelts',
                   'new_car_feature_data_3_list_28_value': 'Hill Assist',
                   'new_car_feature_data_3_list_29_value': 'Electronic Stability Control',
                   'new_car_feature_data_3_list_30_value': 'Hill Descent Control',
                   'new_car_feature_data_3_list_31_value': 'Speed Alert',
                   'new_car_feature_data_3_list_32_value': 'Impact Sensing Auto Door Lock',
                   'new_car_feature_data_3_list_33_value': 'View360Camera',
                   'new_car_feature_data_3_list_34_value': 'Sos Emergency Assistance',
                   'new_car_feature_data_3_list_35_value': 'Geo Fence Alert',
                   'new_car_feature_data_3_list_36_value': 'Blind Spot Monitor',
                   'new_car_feature_data_3_list_45_value': 'Lane Watch Camera'}, inplace = True)
 
df['Day Night Rear View Mirror'] = df['Day Night Rear View Mirror'].apply(lambda aa: 1 if aa == "Day Night Rear View Mirror" else 0)
df['Brake Assist'] = df['Brake Assist'].apply(lambda aa: 1 if aa == "Brake Assist" else 0)
df['Power Door Locks'] = df['Power Door Locks'].apply(lambda ab: 1 if ab == "Power Door Locks" else 0)
df['Child Safety Locks'] = df['Child Safety Locks'].apply(lambda ac: 1 if ac == "Child Safety Locks" else 0)
df['Anti Theft Alarm'] = df['Anti Theft Alarm'].apply(lambda ac: 1 if ac == "Anti Theft Alarm" else 0)
df['Driver Air Bag'] = df['Driver Air Bag'].apply(lambda ad: 1 if ad == "Driver Air Bag" else 0)
df['Passenger Air Bag'] = df['Passenger Air Bag'].apply(lambda ae: 1 if ae == "Passenger Air Bag" else 0)
df['Passenger Side Rear View Mirror'] = df['Passenger Side Rear View Mirror'].apply(lambda af: 1 if af == "Passenger Side Rear View Mirror" else 0)
df['Halogen Headlamps'] = df['Halogen Headlamps'].apply(lambda ag: 1 if ag == "Halogen Headlamps" else 0)
df['Rear Seat Belts'] = df['Rear Seat Belts'].apply(lambda ah: 1 if ah == "Rear Seat Belts" else 0)
df['Seat Belt Warning'] = df['Seat Belt Warning'].apply(lambda ai: 1 if ai == "Seat Belt Warning" else 0)
df['Door Ajar Warning'] = df['Door Ajar Warning'].apply(lambda aj: 1 if aj == "Door Ajar Warning" else 0)
df['Side Impact Beams'] = df['Side Impact Beams'].apply(lambda ak: 1 if ak == "Side Impact Beams" else 0)
df['Front Impact Beams'] = df['Front Impact Beams'].apply(lambda bh: 1 if bh == "Front Impact Beams" else 0)
df['Adjustable Seats'] = df['Adjustable Seats'].apply(lambda bi: 1 if bi == "Adjustable Seats" else 0)
df['Keyless Entry'] = df['Keyless Entry'].apply(lambda bj: 1 if bj == "Keyless Entry" else 0)
df['Centrally Mounted Fuel Tank'] = df['Centrally Mounted Fuel Tank'].apply(lambda bk: 1 if bk == "Centrally Mounted Fuel Tank" else 0)
df['Engine Immobilizer'] = df['Engine Immobilizer'].apply(lambda bl: 1 if bl == "Engine Immobilizer" else 0)
df['Engine Check Warning'] = df['Engine Check Warning'].apply(lambda bl: 1 if bl == "Engine Check Warning" else 0)
df['Crash Sensor'] = df['Crash Sensor'].apply(lambda bm: 1 if bm == "Crash Sensor" else 0)
df['Ebd'] = df['Ebd'].apply(lambda bn: 1 if bn == "Ebd" else 0)
df['Anti Theft Device'] = df['Anti Theft Device'].apply(lambda bo: 1 if bo == "Anti Theft Device" else 0)
df['Rear Camera'] = df['Rear Camera'].apply(lambda r: 1 if r == "Rear Camera" else 0)
df['Speed Sensing Auto Door Lock'] = df['Speed Sensing Auto Door Lock'].apply(lambda r: 1 if r == "Speed Sensing Auto Door Lock" else 0)
df['No Of Airbags'] = df['No Of Airbags'].apply(lambda r: 1 if r == "No Of Airbags" else 0)
df['Automatic Head Lamps'] = df['Automatic Head Lamps'].apply(lambda r: 1 if r == "Automatic Head Lamps" else 0)
df['Isofix Child Seat Mounts'] = df['Isofix Child Seat Mounts'].apply(lambda r: 1 if r == "Isofix Child Seat Mounts" else 0)
df['Pretensioners And Force Limiter Seatbelts'] = df['Pretensioners And Force Limiter Seatbelts'].apply(lambda bn: 1 if bn == "Pretensioners And Force Limiter Seatbelts" else 0)
df['Hill Assist'] = df['Hill Assist'].apply(lambda bn: 1 if bn == "Hill Assist" else 0)
df['Electronic Stability Control'] = df['Electronic Stability Control'].apply(lambda bn: 1 if bn == "Eletronic Stability Control" else 0)
df['Hill Descent Control'] = df['Hill Descent Control'].apply(lambda bn: 1 if bn == "Hill Descent Control" else 0)
df['Speed Alert'] = df['Speed Alert'].apply(lambda bn: 1 if bn == "Speed Alert" else 0)
df['Impact Sensing Auto Door Lock'] = df['Impact Sensing Auto Door Lock'].apply(lambda bn: 1 if bn == "Impact Sensing Auto Door Lock" else 0)
df['View360Camera'] = df['View360Camera'].apply(lambda r: 1 if r == "View360Camera" else 0)
df['Sos Emergency Assistance'] = df['Sos Emergency Assistance'].apply(lambda r: 1 if r == "Sos Emergency Assistance" else 0)
df['Geo Fence Alert'] = df['Geo Fence Alert'].apply(lambda r: 1 if r == "Geo Fence Alert" else 0)
df['Blind Spot Monitor'] = df['Blind Spot Monitor'].apply(lambda r: 1 if r == "Blind Spot Monitor" else 0)
df['Lane Watch Camera'] = df['Lane Watch Camera'].apply(lambda r: 1 if r == "Lane Watch Camera" else 0)


#each column value_counts, which counts is high mention 1 otherwise 0

In [11]:
df.rename(columns={'new_car_feature_data_4_list_0_value': 'Cd Player',
                   'new_car_feature_data_4_list_1_value': 'Radio',
                   'new_car_feature_data_4_list_2_value': 'Speakers Front',
                   'new_car_feature_data_4_list_3_value': 'Speakers Rear',
                   'new_car_feature_data_4_list_4_value': 'Integrated2Din Audio',
                   'new_car_feature_data_4_list_5_value': 'Usb Auxiliary Input',
                   'new_car_feature_data_4_list_6_value': 'Bluetooth',
                   'new_car_feature_data_4_list_7_value': 'Number Of Speaker',
                   'new_car_feature_data_4_list_8_value' : 'Apple Car Play',
                   'new_car_feature_data_4_list_9_value': 'Android Auto',
                   'new_car_feature_data_4_list_10_value': 'Wireless Phone Charging',
                   'new_car_feature_data_4_list_11_value': 'Rear Entertainment System',
                   'new_car_feature_data_4_list_12_value': 'Wifi Connectivity',
                   'new_car_feature_data_4_list_13_value': 'Compass',
                   'new_car_feature_data_4_list_19_value': 'Touch Screen'}, inplace=True)


#each column value_counts, which counts is high mention 1 otherwise 0
df['Cd Player'] = df['Cd Player'].apply(lambda ba: 1 if ba == "Cd Player" else 0)
df['Radio'] = df['Radio'].apply(lambda bb: 1 if bb == "Radio" else 0)
df['Speakers Front'] = df['Speakers Front'].apply(lambda bc: 1 if bc == "Speakers Front" else 0)
df['Speakers Rear'] = df['Speakers Rear'].apply(lambda bd: 1 if bd == "Speakers Rear" else 0)
df['Integrated2Din Audio'] = df['Integrated2Din Audio'].apply(lambda be: 1 if be == "Integrated2Din Audio" else 0)
df['Usb Auxiliary Input'] = df['Usb Auxiliary Input'].apply(lambda bf: 1 if bf == "Usb Auxiliary Input" else 0)
df['Bluetooth'] = df['Bluetooth'].apply(lambda bg: 1 if bg == "Bluetooth" else 0)
df['Number Of Speaker'] = df['Number Of Speaker'].apply(lambda bg: 1 if bg == "Number Of Speaker" else 0)
df['Apple Car Play'] = df['Apple Car Play'].apply(lambda bg: 1 if bg == "Apple Car Play" else 0)
df['Android Auto'] = df['Android Auto'].apply(lambda r: 1 if r == "Android Auto" else 0)
df['Wireless Phone Charging'] = df['Wireless Phone Charging'].apply(lambda r: 1 if r == "Wireless Phone Charging" else 0)
df['Rear Entertainment System'] = df['Rear Entertainment System'].apply(lambda r: 1 if r == "Rear Entertainment System" else 0)
df['Wifi Connectivity'] = df['Wifi Connectivity'].apply(lambda r: 1 if r == "Wifi Connectivity" else 0)
df['Compass'] = df['Compass'].apply(lambda r: 1 if r == "Compass" else 0)
df['Touch Screen'] = df['Touch Screen'].apply(lambda r: 1 if r == "Touch Screen" else 0)


In [12]:
#drop columns in'new_car_specs'
df.drop(columns=['new_car_specs_heading','new_car_specs_top_0_key',
                 'new_car_specs_top_1_key','new_car_specs_top_1_value',#same as Engine displacement
                 'new_car_specs_top_2_key','new_car_specs_top_3_key',
                 'new_car_specs_top_4_key', 'new_car_specs_data_0_heading',
                 'new_car_specs_data_0_subHeading', 'new_car_specs_data_0_list_0_key',    
                 'new_car_specs_data_0_list_1_key','new_car_specs_data_0_list_2_key', #same as engine displacement
                 'new_car_specs_data_0_list_2_value','new_car_specs_data_0_list_3_key',
                 'new_car_specs_data_0_list_4_key', 'new_car_specs_data_0_list_5_key',
                 'new_car_specs_data_0_list_6_key', 'new_car_specs_data_0_list_7_key',
                 'new_car_specs_data_0_list_8_key', 'new_car_specs_data_0_list_9_key',
                 'new_car_specs_data_0_list_10_key', 'new_car_specs_data_0_list_11_key', #same as Turbo and Super charger
                 'new_car_specs_data_0_list_11_value', 'new_car_specs_data_0_list_12_key',
                 'new_car_specs_data_0_list_12_value', 'new_car_specs_data_1_heading', 
                 'new_car_specs_data_1_subHeading', 'new_car_specs_data_2_heading',
                 'new_car_specs_data_2_subHeading', 'new_car_specs_commonIcon',
                 'new_car_specs_data_2_list_2_key', 'new_car_specs_data_2_list_2_value',#same as seats
                 'new_car_specs_data_1_list_0_key', 'new_car_specs_data_1_list_1_key',
                 'new_car_specs_data_1_list_2_key', 'new_car_specs_data_1_list_3_key',
                 'new_car_specs_data_1_list_4_key', 'new_car_specs_data_1_list_5_key',
                 'new_car_specs_data_1_list_6_key', 'new_car_specs_data_1_list_7_key',
                 'new_car_specs_data_2_list_0_key', 'new_car_specs_data_2_list_1_key',
                 'new_car_specs_data_2_list_3_key', 'new_car_specs_data_2_list_4_key',
                 'new_car_specs_data_2_list_5_key', 'new_car_specs_data_2_list_6_key',
                 'new_car_specs_data_2_list_7_key', 'new_car_specs_data_2_list_8_key',
                 'new_car_specs_data_2_list_9_key', 'new_car_specs_data_2_list_10_key',
                 'new_car_specs_data_2_list_11_key', 'new_car_specs_data_1_list_8_key',
                 'new_car_specs_data_1_list_8_value', 'new_car_specs_data_1_list_9_key',
                 'new_car_specs_data_1_list_9_value', 'new_car_specs_data_1_list_10_key',
                 'new_car_specs_data_1_list_10_value', 'new_car_specs_data_1_list_11_key',
                 'new_car_specs_data_1_list_11_value', 'new_car_specs_data_2_list_12_key',
                 'new_car_specs_top_5_key', 'new_car_specs_top_5_value',#same as seats
                 'new_car_feature_data_4_heading', 'new_car_feature_data_4_subHeading'], inplace=True)

In [13]:
df.rename(columns={'new_car_specs_top_0_value': 'Mileage',
                  'new_car_specs_top_2_value': 'Max Power',
                  'new_car_specs_top_3_value': 'Torque',
                  'new_car_specs_top_4_value': 'Wheel Size',
                   'new_car_feature_top_8_value': 'Dvd Player',
                  'new_car_specs_data_0_list_0_value': 'Color',
                  'new_car_specs_data_0_list_1_value': 'Engine Type',
                  'new_car_specs_data_0_list_3_value': 'Max Power_rpm',
                  'new_car_specs_data_0_list_4_value' : 'Max Torque',
                  'new_car_specs_data_0_list_5_value': 'No of Cylinder',
                  'new_car_specs_data_0_list_6_value': 'Values per Cylinder',
                  'new_car_specs_data_0_list_7_value': 'Value Configuration',
                  'new_car_specs_data_0_list_8_value': 'Fuel Supply System',
                  'new_car_specs_data_0_list_9_value': 'Turbo Charger',
                  'new_car_specs_data_0_list_10_value': 'Super Charger',
                   'new_car_specs_data_1_list_0_value': 'Length',
                   'new_car_specs_data_1_list_1_value': 'Width',
                   'new_car_specs_data_1_list_2_value': 'Height',
                   'new_car_specs_data_1_list_3_value': 'Wheel Base',
                   'new_car_specs_data_1_list_4_value': 'Front Tread',
                   'new_car_specs_data_1_list_5_value': 'Rear Tread',
                   'new_car_specs_data_1_list_6_value': 'Kerb Weight',
                   'new_car_specs_data_1_list_7_value': 'Gross Weight',
                   'new_car_specs_data_2_list_0_value': 'Gear Box',
                   'new_car_specs_data_2_list_1_value': 'Drive Type',
                   'new_car_specs_data_2_list_3_value': 'Steering Type',
                   'new_car_specs_data_2_list_4_value': 'Turning Radius',
                   'new_car_specs_data_2_list_5_value': 'Front Brake Type',
                   'new_car_specs_data_2_list_6_value': 'Rear Brake Type',
                   'new_car_specs_data_2_list_7_value': 'Top Speed',
                   'new_car_specs_data_2_list_8_value': 'Acceleration',
                   'new_car_specs_data_2_list_9_value': 'Tyre Type',
                   'new_car_specs_data_2_list_10_value': 'Alloy Wheel Size',
                   'new_car_specs_data_2_list_11_value': 'No of Doors',
                   'new_car_specs_data_2_list_12_value': 'Cargo Volume'}, inplace=True)


In [14]:
# Clean and extract mileage from 'new_car_specs_top_0_value' column 
df['Mileage'] = df['Mileage'].apply(
    lambda mileage: 
    float(str(mileage).replace('kmpl', '').replace('km/kg', '').strip()) 
    if str(mileage).replace('kmpl', '').replace('km/kg', '').strip().replace('.', '', 1).isdigit() and 
    float(str(mileage).replace('kmpl', '').replace('km/kg', '').strip()) < 100 
    else np.nan)

# Extract numeric values ending with 'bhp', otherwise set as NaN
df['Max Power'] = df['Max Power'].str.extract(r'(\d+\.?\d*)bhp').astype(float)

# Extract numeric values ending with 'Nm' or 'nm', otherwise set as NaN
df['Torque'] = df['Torque'].apply(
    lambda x: float(x[:-2].strip()) if isinstance(x, str) and x.lower().endswith('nm') else np.nan
)




In [15]:

# List of seat numbers and their lowercase equivalents
Seat = ['2', '3', '4', '5', '6', '7', '8', '9', '10']
Seat_lower = [Sl.lower() for Sl in Seat]

# Clean the 'Wheel Size' column using a lambda function
df['Wheel Size'] = df['Wheel Size'].apply(
    lambda ws: int(ws[1:]) if isinstance(ws, str) and ws.startswith('R') and ws[1:].isdigit()
    else pd.NA if str(ws).strip() in Seat_lower else ws
)

df['Dvd Player'] = df['Dvd Player'].apply(lambda r: 1 if r == "Dvd Player" else 0)


# color column occurs number to replace null vales
col = ['5', '6', '8']
df['Color'] = df['Color'].apply(
    lambda y: np.nan if str(y).strip().lower() in [i.lower() for i in col] else y)
#computer known color
color = {'Gray': "Grey", 'Starry Black': "Black", 'Golden': 'Gold', 'O Purple': "Purple", 'Other': "Others", 'SILKY SILVER': "Silver", "Beige": "Brown",
         'TITANIUM GREY': "Grey", 'Galaxy Blue': "Blue", 'Gravity Gray': "Grey", "Glacier White Pearl": "White", 'Light Orange': "Orange", 'METALL': "Silver",
         "MODERN STEEL METALLIC": 'Grey', 'PLATINUM WHITE': "White", 'Technometgrn+Gryroof': "Green,Grey", 'Golden Brown': "Gold,Brown", 'Dark Blue': "Blue",
         'Star Dust': "Grey", 'Flash Red': "Red", 'Wine Red': "Red", 'Taffeta White': "White", 'T Wine': "Red" , 'Prime Star Gaze': "Grey", 'BEIGE': "Brown",
         'TAFETA WHITE': "White", 'Minimal Grey': "Grey", 'P Black': "Black", 'Golden brown': "Gold,Brown", 'MET ECRU BEIGE': "Brown,Yellow", 
         'COPPER': "Brown", 'TITANIUM': "Grey", 'CHILL': "Grey", 'Burgundy': "Red,Purple", 'BERRY RED': "Red", 'PREMIUM AMBER METALLIC': "Silver", 
         'R EARTH': "Red,Brown", 'PLATINUM WHITE PEARL': "White", 'Fiery Red': "Red", 'PLATINUM SILVER': "Silver", 'ORCHID WHITE PEARL': "White", 
         'CARNELIAN RED PEARL': "Red", 'POLAR WHITE': "White", 'Lunar Silver Metallic': "Silver", 'Medium Blue': "Blue", 'Alabaster Silver Metallic': "Silver",
         'Carbon Steel': "Grey", 'Cavern Grey': "Grey", 'ESPRESO_BRWN': "Brown", 'Pearl White': "White", 'Polar White': "White", 'Magma Grey': "Grey",
         'GOLDEN BROWN': "Gold,Brown", 'CBeige': "Brown", 'Dark Red': "Red", 'Falsa Colour': "Others", 'Cherry': "Red", 'Sleek Silver': "Silver", 
         'Hip Hop Black': "Black", 'Nexa Blue': "Blue", 'Modern Steel Metallic': "Grey", 'Arctic Silver': "Silver", 'Passion Red': "Red", 'Cirrus White': "White",
         'Arizona Blue': "Blue", 'Silky Silver': "Silver", 'Modern Steel Metal': "Grey", 'magma gray': "Grey", 'Bright Silver': "Silver",'Porcelain White': "White",
         'Goldan BRWOON': "Gold,Brown", 'm grey': "Grey", 'Metallic silky silver': "Silver", 'Mist Silver': "Silver", 'Granite Grey': "Grey", 'Off White': "White", 
         'urban titanim': "Grey", 'g brown': "Brown", 'Solid Fire Red': "Red", 'beige': "Brown", 'Metallic Azure Grey': "Grey", 'Bronze': "Gold,Brown",
         'Rosso Brunello': "Red,Brown", 'Aurora Black Pearl': "Black", 'b grey': "Grey", 'Radiant Red M': "Red", 'Foliage': "Green", 'c brown': "Brown",
         'c bronze': "Gold,Brown", 'Champagne Mica Metallic': "Others", 'Bold Beige Metallic': "Brown", 'Symphony Silver': "Silver", 'Sunset Red': "Red",
         'Metallic Magma Grey': "Grey", 'chill': "Grey", 'Metallic Glistening Grey': "Grey", 'Superior white': "White", 'Silky silver': "Silver", 
         'Urban Titanium Metallic': "Grey", 'Smoke Grey': "Grey", 'Pearl Arctic White': "White", 'Burgundy Red Metallic': "Red", 'b red': "Red", 
         'a silver': "Silver", 'Tafeta White': "White", 'Coral White': "White", 'Diamond White': "White", 'Brick Red': "Red", 'Carnelian Red Pearl': "Red", 
         'Mediterranean Blue': "Blue", 'Candy White': "White", 'Metallic Premium silver': "Silver", 'Glistening Grey': "Grey", 'Super white': "White",
         'Deep Black Pearl': "Black", 'Twilight Blue': "Blue", 'Caviar Black': "Black", 'Pearl Met. Arctic White': "Grey", 'Pure white': "White", 
         'StarDust': "Grey", 'Alabaster Silver Metallic - Amaze': "Grey", 'Ray blue': "Blue", 'OUTBACK BRONZE': "Gold,Brown", 'Moonlight Silver': "Silver", 
         'Fire Brick Red': "Red", 'Cashmere': "Brown",'Pearl Snow White': "White", 'G Brown': "Brown", 'Parpel': "Purple", 'Phantom Black': "Black",
         'Outback Bronze': "Gold,Brown", 'Cherry Red': "Red", 'Silicon Silver': "Silver", 'Daytona Grey': "Grey", 'golden brown': "Gold,Brown", 
         'Sky Blue': "Blue", 'Light Silver': "Silver", 'Out Back Bronze': "Gold,Brown"}
# Replace values based on the dictionary
df['Color'] = df['Color'].replace(color)


et = ['0', '796', '970', '998', '999', '1061', '1086', '1197', '1248', '1298', '1332', '1353', '1364', '1368', '1396', '1399', '1405', '1451', '1461',
      '1462', '1490', '1496', '1498', '1596', '1799', '1950', '1956', '1969', '1984', '1995', '1997', '1998', '1999', '2112', '2446', '2953', '2981',
      '2993', '2998', '3993', 'Power']

df['Engine Type'] = df['Engine Type'].apply(
    lambda y: np.nan if str(y).strip().lower() in [i.lower() for i in et] else y)


In [16]:
rpm = ['3', '4', '15', '16', '110Nm', '113Nm@4400rpm', '115Nm@4000rpm', '115Nm@4500rpm','70Nm@1000rpm', '190bhp', '96Nm@3500rpm', 
       '121.5Nm@4200rpm', '122Nm@4400-4800rpm', '136.8Nm@4400rpm', '142Nm@4000rpm', '170Nm', '171.6Nm@1500-4000rpm', '219.6Nm@1500-2750rpm',
       '220Nm@1500-2700rpm', '220Nm@1500-2750rpm','242nm@1500-3200rpm', '245Nm', '245Nm@1750rpm','250Nm', '250Nm@1600-3600rpm', '98bhp',
      '160 BHP', '400bhp', '300Nm', '235bhp','360Nm@1500â€“2500rpm','400nm@1750â€“2500rpm','620Nm@1500â€“2500rpm','250Nm@1620â€“4000rpm',
      '70Nm@1000rpm', '760Nm', '84.33Nm@4500 rpm', '96Nm@3500rpm', '98.5Nm@4300rpm', '300Nm', '320Nm@1500-4100rpm', '350Nm@1450-4800rpm', 
      '395Nm', '400Nm@1500-4000rpm', '400Nm@1600-2600rpm', '242nm@1500-3200rpm', '270Nm@1000rpm', '280Nm', '280Nm@1350-4600rpm']


df['Max Power_rpm'] = df['Max Power_rpm'].apply(
    lambda y: np.nan if str(y).strip().lower() in [i.lower() for i in rpm] else y)

# Function to extract only RPM
def extract_rpm(value):
    if pd.isna(value):  # Check if the value is NaN
        return None
    
    # Regular expression to capture RPM, accounting for commas
    match = re.search(r'@?\s?(\d{1,4}(?:,\d{3})*)\s*rpm', value, re.IGNORECASE)
    if match:
        rpm = match.group(1).replace(',', '')  # Remove comma from RPM
        return rpm
    return None

# Applying the function to create the 'RPM' column
df['Max Power_rpm'] = df['Max Power_rpm'].apply(extract_rpm)

# Converting 'RPM' to a numeric type
df['Max Power_rpm'] = pd.to_numeric(df['Max Power_rpm'], errors='coerce')

#remove undefined in no of cylinder
un = ['undefined']
df['No of Cylinder'] = df['No of Cylinder'].replace({i: np.nan for i in un}, regex=True)


In [17]:
# Define a list of invalid values
mt = ['3', '4', '6', 'undefined', '400Nm', '300Nm']

# Convert all values to string and clean up 'Max Torque' column
df['Max Torque'] = df['Max Torque'].astype(str).apply(
    lambda x: x.strip() if x.strip().lower() not in [i.lower() for i in mt] else np.nan
)

# Function to extract torque value and engine speed
def extract_torque_and_speed(value):
    # Extract torque value
    torque_match = re.search(r'(\d+\.?\d*)\s*(Nm|kgm|KGM|nm|NM)?', value, re.IGNORECASE)
    # Extract engine speed
    speed_match = re.search(r'@(\d+)', value)
    
    torque = float(torque_match.group(1)) if torque_match else np.nan
    speed = int(speed_match.group(1)) if speed_match else np.nan
    
    return speed if not np.isnan(speed) else torque

# Apply the extraction function to the 'Max Torque' column
df['Max Torque'] = df['Max Torque'].apply(lambda x: extract_torque_and_speed(str(x)) if pd.notna(x) else np.nan)


In [18]:
#remove VAL in Values per Cylinder
VAL = ['GDi', 'Yes', 'No', 'MPFI', 'MPFi', 'DOHC', 'MPFI ', 'CRDi', 'Twin', 'twin', 'Electric', 
       'Multipoint Injection', 'Turbo', 'SOHC', '16 Modules 48 Cells']
df['Values per Cylinder'] = df['Values per Cylinder'].replace({j: np.nan for j in VAL}, regex = True)

conf = ['DOHC', 'SOHC', 'undefined', 'DOHC ', '16-valve DOHC layout', 'iDSI', '23 Modules 69 Cells', 'SOHC ', 'DOHC with VIS', 
        'DOHC with VGT', 'DOHC with TIS']
df['Value Configuration'] = df['Value Configuration'].apply(lambda y: y if y in conf else np.nan)
df['Value Configuration'] = df['Value Configuration'].replace({'DOHC ': "DOHC", 'SOHC ': "SOHC"})


In [19]:
fss = ['69.6 x 82 mm', '84.0 x 90.0 mm', '69 x 72 mm', 'No', 'Yes','82 x 94.6 mm', '16.0:1', '84 x 90 mm', '74 x 85', '12.0:1', 
       '82.0 x 78.8 mm','76.5 X 75.6 mm', '73 x 72 mm','73 X 71.5 mm','74.0 x 75.5 mm', '71.5 x 83.5 mm', '88.9 X 101.6 mm', 
       '78 X 83mm', '73.5 mm x 73.5 mm', ':1', '84 X 90 mm', '16.2:2', '11.0:1', '10.5:1', '73.0 X 79.5 mm', '74 x 75.5 mm', 
       '78 x 83.6 mm', '69.6 X 82 mm', '2.99 X 3.46 mm', '94 x 94 mm', '9.3:1','76.5 x 87 mm', '82 x 93.2 mm', '76 x 80.5 mm', 
       '13.0 : 1', '74.0 x 85.0 mm', '81 x 93.15 mm', '16.8 : 1', '73x71.5', '68.5 x 72 mm', '98.0 x 90.5 mm', '10.7:1', 'yes', 
       '11.04:1', '94.6mmX82.0mm', '81 X 95.5 mm', '12.0 ±0.3', '74.5 x 76.4', '68.5 x 72.0 mm', '10.7:1 ', '16.2:1', '81 x 88 mm',
       '83 x 92 mm', '94 X 90 mm', 'Twin', '66 x 77 mm', '74 x 71.5 mm','89.0 X 88.3 mm', '8.5:1', '87.5 X 83.1 mm', '96 (mm) / 83 (mm)', 'no',
       '96 (mm) / 83 (mm)']

df['Fuel Supply System'] = df['Fuel Supply System'].replace({k: np.nan for k in fss}, regex = True)

df['Fuel Supply System'] = df["Fuel Supply System"].replace({ 'MPFi': "MPFI", 'Direct Injection': "CRDI", 'Electronic Injection System': "EFI",
                                                             'PGM - Fi': "PGM-FI", 'Direct Injection ': "CRDI", 'PGM-Fi': "PGM-FI", 'CDI': "CRDI",
                                                             'Multi Point Fuel Injection': "MPFI", 'CRDi': "CRDI", 'Common-Rail type': "CRDI", 'GDi': "GDI",
                                                             'Multipoint Injection': 'MPFI', 'common rail system': "CRDI", 'EFI': "EFI", 'EFi': "EFI", 'MPi': "MPI",
                                                             'Common Rail Direct Injection (dCi)': "CRDI", 'EFI(Electronic Fuel Injection)': "EFI", 'TDCi': "TDCI", 
                                                             'Electronic Fuel Injection': "EFI", 'Common Rail': "CRDI", 'Multi point Fuel Injection': "MPFI",
                                                             'Direct Injection Common Rail': "CRDI", 'MPFI ': "MPFI", 'PGM-FI (Programmed Fuel Injection)': "PGM-FI",
                                                             'CRDI ': "CRDI", 'EFI (Electronic Fuel Injection)': "EFI", 'PGM-FI (Programmed Fuel Inje': "PGM-FI",
                                                             'PFI': "PGM-FI", 'Electronic Fuel Injection(EFI)': "EFI",  'Direct Injection Common Rail ': "CRDI",
                                                             'Common Rail Direct Injection': "CRDI", 'Common-Rail Type': "CRDI", 'MPFi ': "MPFI", 'Common Rail ': "CRDI",
                                                             'Multi-point injection': "MPI", 'Common rail direct injection': "CRDI", 'Common Rail Injection': "CRDI",
                                                             'PGM-FI (Programmed Fuel Inject': "PGM-FI", 'Direct Fuel Injection': "CRDI",
                                                             'EFI (Electronic Fuel Injection': "EFI",  'Gasoline Direct Injection': "GDI",})
        
      
       

In [20]:
tc = ['73 X 82 mm', '79 X 76.5 mm', '77 X 85.8 mm', '74 x 85 mm', '73.0 X 89.4 mm', '16.0:1', '10.5:1', '74 X 85 mm',
      '81.0mm X 87.2mm', '11', '68.5 x 72.0 mm', '83 x 92 mm', '76x82.5', '71.5 x 81.5 mm', '69.6 X 82mm', '82.5 x 84.2 mm', 
      '82 x 93.2 mm', '76.5 x 86.9 mm', '71 x 75.6 mm', ':1', '11.0:1', '16.5:1', '69.6 X 82 mm', '74.5 x 76.4', '77 X 80.5 mm', 
      '69.6 X 82', '76.0 X 82.5 mm', '16.2+/-2:1', '73.5 X 88.3 mm', '73 x 82mm', '75.0 x 55.0 mm', '81.5 x 95.8 mm', '74.5x76.4 mm', 
      '82.5 x 92.8', '73 x 72 mm', '75.5 X 83.5 mm', '73 X 71.5 mm', '75 x 90 mm', '77.0 X 85.44 mm', '73 X 82mm', '79.5 x 95.5 mm', 
      '69.6 x 82 mm', '71 X 75.6 mm', '74.5 X 76.4mm', '10.0:1', '69 x 72 mm', '77.2mm X 84.5mm', '16:01', '77.0mm X 85.44mm', '83 X 92.35 mm',
      '72.2 x 81.3', '78.3 x 86.4 mm', '76.5 X 86.9 mm', '85 x 96 mm', '92.9 x 86.0 mm', '83.0 x 92.0 mm', '77x85.8', '77.2 X 84.5 mm', '82.5 x 92.8 mm',
      '79.5 x 80.5 mm', '3.39 in X 3.39 mm', '77.0 x 80.5 mm', '09:01', '10:01', '84 X 90 mm', '3.62 X 4.08 mm', '88.5 X 96.0 mm', '75 X 79.5 mm', 
      '74 x 75.5 mm', '9.4:1', '71 x 84.1', '73.0 x 89.4 mm', '81 x 95.5 mm', '68.5 x 67.5 mm', '69.9X82 mm', '76 x 80.5 mm', '73.5 mm x 73.5 mm',
      '78 x 83.6', '76.0 x 76.5 mm', '73.5mm X 73.5mm', '75 x 79.5 mm', '75 X 79 mm', '76 X 80.5 mm', '75.5 x 83.5 mm', '73.7 x 82.0 mm',
      '76 X 80.5mm', '70 x 72 mm', '2.85 X 3.57 mm', '81.0 x 95.5 mm', '79 X 76.4 mm', '94 X 90 mm', '73  x 79.5 mm', '11.2:1', '82.5 x 84.1 mm', 
      '15.8:1', '78 X 83.6 mm', '11:01', '78 x 83 mm', '9.9:1', '84.5 x 89.0', '84.0 x 90.0 mm', '77 x 80.5 mm', '81 x 87.3 mm', '75 X 79mm', 
      '74.5 x 76.4 mm', '9.8:1', '17.6:1', '81 x 77 mm', '77.0 X 85.1 mm', '18.5:1', '69.9 x 82 mm', '77.9 x 73.4 mm', '83x92.4mm', '74.0 x 75.5 mm',
      '95.4X104.9', '16,5 :1', '81.6 x 86 mm', '88 x 97 mm', '81X77mm', '84 x 88 mm', '90 X 98 mm', '78.0 X 78.4mm', '17.9:1', '74.5 x 81 mm', '17.7:1',
      '73 x 79.5 mm', '10', '83.0 x 92.0mm', '9.0:1', '15.5:1', '9.5:1', '90.0 X 84.4 mm', '74.5 x 80 mm', '92 X 90 mm', '81 X 87.3 mm', '79 x 81.5 mm', 
      '66 x 77 mm', '18.0:1', '10.7:1', '9.75:1', '9.6:1', '18:01', '81.0 X 87.2mm', '81X93.15 mm', '81 x 95.5', '77.0mm X85.44mm', '75 x 87 mm',
      '71.0 X 75.6', '77.0*80.4 mm', '81.0 x 86.4 mm', '77.0 X 80.5', '71.5 x 83.5 mm', '92 x 92 mm', '79 X 91.5 mm', '78 X 83 mm','95.0 x 100.0 mm',
      '3.35 X 3.81 mm', '81 X 93.2 mm', '68.5 x 78 mm', '97 X 100 mm', '73.0 X 79.5 mm', '83.5 x 73.5 mm','8.9:1', '81 X 95.5 mm', '69.9X82', 
      '83x92.3mm', '12.5:1', '10.3:1', '83 X 92 mm', '16.8:1', 'Twin', '77.0*80.4 mm', '77.0*80.4 mm']

df['Turbo Charger'] = df['Turbo Charger'].replace({l: np.nan for l in tc}, regex = True)
df['Turbo Charger'] = df['Turbo Charger'].replace('no', 0)
df['Turbo Charger'] = df['Turbo Charger'].replace('No', 0)
df['Turbo Charger'] = df['Turbo Charger'].replace('yes', 1)
df['Turbo Charger'] = df['Turbo Charger'].replace('Yes', 1)

In [21]:
sc = ['09:01', '10:01', '16:01', '18:01', '21:01', '22:01', '9.01', '9.8', '10.1', '17.9', '10.0:1', '10.1:1', '10.2:1', '10.3.1', '10.5:1', '10.5;1',
      '10.7:1', '10.8:1', '11.0:1', '11.3:1', '15.0:1', '15.5:1', '15.8:1', '16.0:1', '16.1:1', '16.2:1', '16.5 :1', '16.5:1', '17.2:1', '17.3:1', '17.5:1',
      '17.6:1', '17.7:1', '17.8:1', '17.9:1', '18.0:1', '18.5:1', '19.0:1', '19.5 : 1', '19.5:1', '22.2:1', '8.8:1', '8.9:1', '81 x 87.3 mm', '9.0:1',
      '9.3:1', '9.5: 1', '9.5:1', '9.6:1', '9.75:1', '9.8:1', '9.5 : 1']

df['Super Charger'] = df['Super Charger'].replace({l: np.nan for l in sc}, regex = True)
df['Super Charger'] = df['Super Charger'].replace('NO', 0)
df['Super Charger'] = df['Super Charger'].replace('Yes', 1)
df['Super Charger'] = df['Super Charger'].replace('No', 0)


# List of values to replace with NaN
len = ['5', '6', '7', '8', '1085kg','1070','870', '825', '875', '1720kg','920kg','1100kg','895kg', 
       'Six Speed Manual with Paddle Shifter', '2560', '1527kg', '1200kg','1050', '5 Speed', '800kg']

# Replace specified values with NaN using lambda
df['Length'] = df['Length'].apply(lambda x: np.nan if str(x) in len else x)

# Extract numeric part using lambda
df['Length'] = df['Length'].apply(lambda x: 
                                int(''.join(filter(str.isdigit, str(x)))) if any(char.isdigit() for char in str(x)) else x)

wid = ['1955kg', 'All Wheel Drive', 'FWD', 'Power', '2225kg', '1494mm', '3200']

# Replace specified values with NaN using lambda
df['Width'] = df['Width'].apply(lambda x: np.nan if str(x) in wid else x)
# Extract numeric part using lambda
df['Width'] = df['Width'].apply(lambda x: 
                                int(''.join(filter(str.isdigit, str(x)))) if any(char.isdigit() for char in str(x)) else x)

ht = ['Tubeless,Radial', 'Tubeless', '3165mm', '5']
# Replace specified values with NaN using lambda
df['Height'] = df['Height'].apply(lambda x: np.nan if str(x) in ht else x)

# Extract numeric part using lambda
df['Height'] = df['Height'].apply(lambda x: 
                                int(''.join(filter(str.isdigit, str(x)))) if any(char.isdigit() for char in str(x)) else x)

In [22]:
wb = ['13', '14', '15', '16', '160', '165', '170', '180', '205', '690', '699', '765', '1365', '1515', '1520', '1527', '1547',
      '1645', '1668', '1,240 kg', '110mm', '1121kg', '116mm', '120mm', '1233kg', '1240kg', '140mm', '1440mm', '145mm', '155mm', 
      '156mm', '1624mm', '170mm', '175mm', '178mm', '180mm', '184mm', '185mm', '190mm', '192mm', '200mm', '205mm', '209mm', '210mm',
      '210 mm', '91mm', 'Power']

# Replace specified values with NaN using lambda
df['Wheel Base'] = df['Wheel Base'].apply(lambda x: np.nan if str(x) in wb else x)

# Extract numeric part using lambda
df['Wheel Base'] = df['Wheel Base'].apply(lambda x: 
                                int(''.join(filter(str.isdigit, str(x)))) if any(char.isdigit() for char in str(x)) else x)


In [23]:
ft = ['3', '4', '5', '625', '675', '690', '710', '725', '730', '745', '750', '755', '765', '770', '815', '825', '835', '845', '855', '875',
      '935', '939', '940', '950', '960', '965', '970', '980', '982', '990', '995', '1010', '1014', '1015', '1020', '1025', '1030', '1036', '1039',
      '1040', '1045', '1050', '1060', '1063', '1065', '1066', '1070', '1080', '1085', '1087', '1090', '1095', '1100', '1102', '1105', '1106', '1110', 
      '1115', '1120', '1125', '1130', '1135', '1140', '1148', '1150', '1160', '1170', '1175', '1180', '1188', '1190', '1192', '1195', '1198', '1200',
      '1201', '1208', '1210', '1211', '1215', '1218', '1220', '1225', '1228', '1230', '1232', '1234', '1235', '1238', '1240', '1245', '1248', '1250', 
      '1255', '1258', '1260', '1265', '1269', '1270', '1275', '1277', '1280', '1285', '1288', '1290', '1295', '1,300', '1300', '1305', '1,310', '1310',
      '1315', '1320', '1330', '1335', '1340', '1345', '1350', '1351', '1355', '1360', '1375', '1395', '1400', '1404', '1405', '1410', '1420', '1425', 
      '1430', '1435', '1440', '1445', '1450', '1455', '1460', '1465', '1468', '1470', '1475', '1480', '1485', '1490', '1495', '1497', '1500', '1505',
      '1510', '1515', '1520', '1525', '1530', '1535', '1537', '1540', '1545', '1547', '1550', '1560', '1565', '1570', '1575', '1580', '1585', '1595', 
      '1615', '1620', '1625', '1628', '1630', '1635', '1640', '1645', '1650', '1655', '1660', '1670', '1675', '1680', '1685', '1690', '1695', '1700', '1705',
      '1710', '1725', '1730', '1735', '1745', '1750', '1755', '1765', '1770', '1775', '1785', '1790', '1795', '1800', '1810', '1820', '1830', '1840', '1850',
      '1855', '1860', '1865', '1870', '1880', '1885', '1890', '1900', '1915', '1920', '1925', '1935', '1940', '1945', '1950', '1960', '1970', '1980', '1990',
      '1994', '2000', '2010', '2020', '2027', '2030', '2040', '2045', '2080', '2095', '2150', '2160', '2186', '2195', '2210', '2220', '2230', '2235', '2245',
      '2250', '2270', '2280', '2360', '2370', '2376', '2440', '2450', '2500', '2560', '2600', '2673', '2700', '2865', '2900', '1,240kg', '1,750 kg', '1010kg',
      '1015Kg', '1016-1026kg', '1016-1033kg', '1016-1043kg', '1020kg', '1021kg', '1023kg', '1026-1044', '1029kg', '1030kg', '1031Kg', '1036Kg', '1039kg', 
      '1040kg', '1042 Kg', '1042kg', '1044kg', '1045kg', '1048kg', '1050kg', '1053-1080kg', '1055-1060kg', '1055kg', '1058kg', '1060kg', '1062-1080', '1062Kg',
      '1064 Kg', '1065-1070kg', '1065kg', '1066 Kg', '1072Kg', '1075kg', '1080kg', '1081-1094', '1084kg', '1085-1095kgs', '1085kg', '1090kg', '1095kg', '1100 kg',
      '1100 - 1125kg', '1100kg', '1103kg', '1105-1110kg', '1105kg', '1107-1153', '1110-1130', '1110-1130kg', '1110Kg', '1115-1130', '1115Kg', '1120kg', '1121kg',
      '1124kg', '1125kg', '1130-1170', '1130kg', '1131kg', '1132-1160 kg', '1135-1150', '1135Kg', '1140 kg', '1140kg', '1145kg', '1146kg', '1150 - 1185', 
      '1150 - 1185kg', '1150-1205', '1150kg', '1152kg', '1154kg', '1155 kg', '1155kg', '1160Kg', '1161Kg', '1165kg', '1168kg', '1173kg', '1175kg', '1180kg',
      '1182kg', '1185kg', '1190-1210', '1190kg', '1195kg', '1198kg', '1200-1210kg', '1200kg', '1205kg', '1206kg', '1210 kg', '1210kg', '1212Kg', '1215Kg', '1217kg'
      '1220kg', '1225Kg', '1230Kg', '1231Kg', '1235kg', '1239Kg', '1240kg', '1242Kg', '1245kg', '1246kg', '1250-1255', '1255kg', '1261Kg', '1265kg', '1268Kg',
      '1274kg', '1275Kg', '1277kg', '1280Kg', '1281Kg', '1289kg', '1290 - 1295', '1296Kg', '1300Kg', '1302Kg', '1304Kg', '1306kg', '1309Kg', '1320Kg',
      '1335 kg', '1345kg', '1355 kg', '1365-1372kg', '1389 kg', '1389kg', '1400kg', '1403kg', '1435mm', '1445kg', '1505kg', '1520kg', '1537 kgs', '1543kg',
      '1545kg', '1551 kgs', '1562 kgs', '1585kg', '1598kg', '1608kg', '1615 Kg', '1615kg', '1618mm', '1627mm', '1640kg', '1641 kgs', '1644kg', '1650kg',
      '1655kg', '1660kg', '1680kg', '1700Kg', '1740 Kg', '1760kg', '1765kg', '1770 Kg', '1770kg', '1773kg', '1796kg', '1816kg', '1825kg', '1830kg', '1890kg',
      '1900kg', '1905kg', '1930kg', '2000kg', '2060kg', '2110kg', '2348mm', '2422mm', '2450mm', '2498mm', '2500 mm', '2500mm', '2510mm', '2552mm', '2590mm',
      '2610mm', '2651mm', '2673 mm', '2673mm', '2680 mm', '2688mm', '2740 mm', '2740mm', '2750mm', '2760 mm', '2791mm', '2841mm', '3150mm', '3264mm', '4.6 m',
      '5.3 Meter', '635kg', '650kg', '665kg', '710kg', '726-767', '765 kg', '765kg', '805-825kg', '805kg', '830-845kg', '830kg', '831-854', '840-865', '845kg',
      '855Kg', '865Kg', '890kg', '895kg', '900kg', '905-910kg', '905kg', '907-934', '915kg', '917kg', '920-955', '920kg', '921kg', '924kg', '930kg', '935kg',
      '940kg', '942kg', '945-957', '945kg', '950kg', '955kg', '960Kg', '965Kg', '982kg', '985kg', '990 - 1006', '990kg', '965Kg', '982kg', '985kg', '990 - 1006',
      '990kg', '993kg', '995kg', 'Kerb Weight']
    

# Replace specified values with NaN using lambda
df['Front Tread'] = df['Front Tread'].apply(lambda x: np.nan if str(x) in ft else x)

# Extract numeric part using lambda
df['Front Tread'] = df['Front Tread'].apply(lambda x: 
                                int(''.join(filter(str.isdigit, str(x)))) if any(char.isdigit() for char in str(x)) else x)

rt = ['1000kg', '1315kg', '1405kg', '1510 mm', '1520kg', '1539mm', '1540 mm', '1650kg', '1660Kg', '1670kg', '1690Kg',
      '1705Kg', '2015 kg', '2555kg', '985kg', 'Ventilated Disc', 'Ventilated Discs']

# Replace specified values with NaN using lambda
df['Rear Tread'] = df['Rear Tread'].apply(lambda x: np.nan if str(x) in rt else x)

# Extract numeric part using lambda
df['Rear Tread'] = df['Rear Tread'].apply(lambda x: 
                                int(''.join(filter(str.isdigit, str(x)))) if any(char.isdigit() for char in str(x)) else x)



In [24]:
kb = ['1335', '1405', '1535', '1567', '1670', '1700', '2680', '1445mm', '1496mm', '1510kg', '1512mm', '1514mm', '1520 mm', '1530mm', '1530 mm', '1534mm',
      '1540mm', '1570mm', '1630kg', '1640 kg', '1660 kg', '1660kg', '1670kg', '1680kg', '1685kg', '1700kg', '1710Kg', '1750Kg', '1770kg', '2030kg', '2060kg',
      '2098kg', '2345Kg', '2449Kg', '2493kg', '2530kg', '3350Kg', '3490Kg', 'Drum', 'Drum in Discs']
# Replace specified values with NaN using lambda
df['Kerb Weight'] = df['Kerb Weight'].apply(lambda x: np.nan if str(x) in kb else x)



# Lambda function to clean and extract the highest number from a range or a single number
df['Kerb Weight'] = df['Kerb Weight'].apply(
    lambda x: (
        float(max(map(float, re.findall(r'\d+', str(x))))) if '-' in str(x) 
        else float(re.sub(r'[^\d]', '', str(x))) 
        if re.sub(r'[^\d]', '', str(x)) != '' 
        else None
    )
)

In [25]:
gw = ['921', '1070', '1275', '1315', '1335', '1355', '1375', '1395', '1012kg', '1180kg', '1190kg', '1210kg', '1220 Kg', '1245kg', 
      '1252Kg', '1260kg', '130 kmph', '1305Kg', '1330kg', '1340kg', '1376kg', '1790kg', '1815kg', '1820kg', '1825kg', '1835kg', 
      '1840kg', '1855kg', '1860kg', '1865kg', '1880kg', '1895kg', '190kmph', '1920kg', '882kg' '886kg', '913kg']
# Replace specified values with NaN using lambda
df['Gross Weight'] = df['Gross Weight'].apply(lambda x: np.nan if str(x) in gw else x)

# Define a lambda function to clean the weight values
def clean_weight(x):
    if pd.isna(x):
        return ''
    return re.sub(r'\s*kg\s*', '', str(x), flags=re.IGNORECASE)

# Apply the lambda function to the DataFrame
df['Gross Weight'] = df['Gross Weight'].apply(clean_weight)

# Convert to numeric, errors='coerce' will convert non-numeric values to NaN
df['Gross Weight'] = pd.to_numeric(df['Gross Weight'], errors='coerce')

In [26]:
#remove gb in gear box
gb = ['4', '5', '5.2 m', 'AWD', 'RWD']
df['Gear Box'] = df['Gear Box'].apply(lambda x: np.nan if str(x) in gb else x)

df['Gear Box'] = df['Gear Box'].astype(str)

df['Gear Box'] = df['Gear Box'].apply(lambda x: re.sub(r'-?[Ss]peed[s]?\b', '', x, flags=re.IGNORECASE).replace('-', '').strip())

df['Gear Box'] = df['Gear Box'].map({'5': "5", 'SPEEDSHIFT TCT 9G': "SPEEDSHIFT DCT 9G", '4': "4", '6': "6", '8 Automatic Transmission': "8 AT", '7': "7",
                                     '9': "9", '8': "8", '8 Steptronic': "8 Steptronic", '8  Tiptronic': "8 Tiptronic", '9G TRONIC': "9G TRONIC", 
                                     '6 AutoSHIFT': "6 AT", 'CVT': "CVT", '6 CVT': "6 CVT", '7 DCT': "7 DCT", '7  DSG': "7 DSG", '7 Stronic': "7 S tronic",
                                     '8 Steptronic Automatic': "8 Steptronic", '10 Automatic': "10 AT", '8 DCT': "8 DCT", '7GDCT': "7 GDCT", '5`': "5",
                                     '8GDCT': "8 GDCT", '7  CVT': "7 CVT", 'Five  Manual': "5 MT", '9GTRONIC': "9G TRONIC", '5  CVT': "5 CVT", 'IVT': "IVT",
                                     '8 Steptronic Sport Automatic Transmission': "8 Steptronic", '8 tiptronic': "8 Tiptronic", '7  DCT': "7 DCT",
                                     '6 DCT': "6 DCT", '9  Tronic': "9G TRONIC", '6 IVT': "6 IVT", '6 iMT': "6 IMT", '7 STronic': "7 S tronic",  'Five': "5",
                                     '7G DCT 7 Dual Clutch Transmission': "7 DCT", '6  MT': "6 MT", '6 AT': "6 AT", 'Direct Drive': "Direct Drive", 
                                     '5  AT+ Paddle Shifters': "5 AT", '10': "10", 'Single transmission': "1 T", 'AGS': "AGS", '9 automatic': "AT",
                                     '6  AT': "6 AT", 'Five  Manual Transmission': "5 MT", 'Six  Manual': "6 MT", 'iMT': "IMT", '6 IMT': "6 IMT", 
                                     'Five  Manual Transmission Gearbox': "5 MT", 'ECVT': "ECVT", '7  7GDCT': "7 GDCT", '5  manual': "5 MT",
                                     'Six  Automatic Transmission': "6 AT", 'Six  Manual with Paddle Shifter': "6 MT", '7 DSG': "7 DSG",
                                     '5  Forward, 1 Reverse': "5  Forward, 1 Reverse", 'Six  Geartronic, Six  Automati': "6 GAT", '6  IVT': "6 IVT",
                                     'Single': '1', '5  Manual (Cable Type Gear Shift)': "5 MT", 'Six  Automatic Gearbox': "6 AT", '6  IMT': "6 IMT",
                                     'Fully Automatic': "Fully Automatic", 'Six   Gearbox': "6", '7  S tronic': "7 S tronic", '6  automatic': "6 AT", 
                                     'Mercedes Benz 7  Automatic': "7 AT", '7  Steptronic Sport': "7 Steptronic", 'Single  reduction gear': "1", 
                                     '7 S tronic': "7 S tronic", 'eCVT': "ECVT", '8  CVT': "8 CVT", '6  iMT': "6 IMT", '9GTRONIC automatic': "9G TRONIC",
                                     '6  with Sequential Shift': "6 Sequential Shift", '7 Steptronic': "7 Steptronic", 'AMG 7 DCT': "AMG 7 DCT",
                                     'AMG SPEEDSHIFT DCT 8G': "AMG SPEEDSHIFT DCT 8G",  '7 PDK': "7 PDK", '8  Sport': "8 Sport",'7 DCT Steptronic': "7 Steptronic"})




#store dt in Drive Type
dt = ['FWD', 'FWD ', '4WD', 'RWD', '2WD', '2 WD', 'AWD', 'AWD INTEGRATED MANAGEMENT', 'Front Wheel Drive', 'Rear Wheel Drive with ESP', '4x2', '4x4', 
       '2WD ', '4X2', '4X4', 'Two Wheel Drive', 'All Wheel Drive', 'RWD(with MTT)', 'Permanent all-wheel drive quattro', '4 WD']
df['Drive Type'] = df['Drive Type'].apply(lambda y: y if y in dt else np.nan)

df['Drive Type'] = df['Drive Type'].map({'FWD': "FWD", 'FWD ': "FWD", '4WD': "4WD", 'RWD': "RWD", '2WD': "2WD", '2 WD': "2WD", 'AWD': "AWD",
                                         'AWD INTEGRATED MANAGEMENT': "AWD, INTEGRATED MANAGEMENT", 'Front Wheel Drive': "FWD",
                                         'Rear Wheel Drive with ESP': "RWD,ESP", '4x2': "4x2", '4x4': "4x4", '2WD ': "2WD", '4X2': "4x2", '4X4': "4x4", 
                                         'Two Wheel Drive': "2WD", 'All Wheel Drive': "AWD", 'RWD(with MTT)': "RWD,MTT",'4 WD': "4WD",
                                         'Permanent all-wheel drive quattro': "Permanent all-wheel drive quattro" })



In [27]:
#electric vs electrical vs electronic belongs to electric bcz electricity converts to other energy
st = ['Power', 'Electric', 'electric', 'EPAS', 'Electrical', 'Electronic', 'Manual', 'power', 'Hydraulic']

df['Steering Type'] = df['Steering Type'].apply(lambda y: y if y in st else np.nan)
    
df['Steering Type'] = df['Steering Type'].map({'Power': "Power", 'Electric': "Electric", 'electric': "Electric", 'EPAS': "EPAS", 'Electrical': "Electric", 
                                               'Electronic': "Electric", 'Manual': "Manual", 'power': "Power", 'Hydraulic': "Hydraulic"})

In [28]:
fbt = ['Ventillated Disc', 'Ventilated disc', 'ABS', 'Vacuum assisted hydraulic dual circuit w', 'Ventilated discs', 'Vantilated Disc',  'Ventlated Disc',
       'Ventilated Disk', 'disc', 'Ventilated Disc', 'Ventilated DIsc', 'Ventilated Disc', 'Disc',' Disc', 'Disc ', 'Ventilated Discs', 'Ventilated Disc ',
       'Solid Disc', 'Disc & Caliper Type', 'Multilateral Disc', 'ventilated Disc']
df['Front Brake Type'] = df['Front Brake Type'].apply(lambda y: y if y in fbt else np.nan)

df['Front Brake Type'] = df['Front Brake Type'].map({'Ventillated Disc': "Ventilated disc", 'Ventilated disc': "Ventilated disc", 'ABS': "ABS", 
                                                     'Vacuum assisted hydraulic dual circuit w': "Vacuum assisted hydraulic dual circuit w", 
                                                     'Ventilated discs': "Ventilated disc", 'Vantilated Disc': "Ventilated disc",  
                                                     'Ventlated Disc': "Ventilated disc", 'Ventilated Disk': "Ventilated disc", 'disc': "Disc", 
                                                     'Ventilated Disc': "Ventilated disc", 'Ventilated DIsc': "Ventilated disc", 'Disc': "Disc",
                                                     'Ventilated Disc': "Ventilated disc", ' Disc': "Disc", 'Disc ': "Disc", 
                                                     'Ventilated Discs': "Ventilated disc", 'Ventilated Disc ': "Ventilated disc",
                                                     'Solid Disc': "Solid disc", 'Disc & Caliper Type': "Disc,Caliper", 'Multilateral Disc': "Multilateral disc",
                                                     'ventilated Disc': "Ventilated disc"})

In [29]:
rbt = ['Disc', 'Disc & Caliper Type', 'Disc & Drum', 'Discs', 'Drum', 'Drum in disc', 'Drum in Discs', 'Drums', 'Leading & Trailing Drum', 
       'Leading-Trailing Drum', 'Self Adjusting Drum', 'self adjusting drums', 'Self-Adjusting Drum', 'Solid Disc', 'Ventilated discs', 
       'Ventilated Disc ', 'Ventilated Disc', 'Solid Disc ','Disc ','Ventialte Disc', 'Ventilated Drum', 'Drum ',
       'Vacuum assisted hydraulic dual circuit w']
#store rear brake type values only
df['Rear Brake Type'] = df['Rear Brake Type'].apply(lambda y: y if y in rbt else np.nan)

#mapping function
df['Rear Brake Type'] = df['Rear Brake Type'].map({'Disc': "Disc", 'Disc & Caliper Type': "Disc,Caliper", 'Disc & Drum': "Disc,Drum", 
                                                   'Discs': "Disc", 'Drum': "Drum", 'Drum in disc': "Disc,Drum", 'Drum in Discs': "Disc,Drum",
                                                   'Drums': "Drum", 'Leading & Trailing Drum':"Leading,Trailing Drum", 
                                                   'Leading-Trailing Drum': "Leading,Trailing Drum", 'Self Adjusting Drum': "Self Adjusting Drum", 
                                                   'self adjusting drums': "Self Adjusting Drum", 'Self-Adjusting Drum': "Self Adjusting Drum", 
                                                   'Solid Disc': "Solid Disc", 'Ventilated discs': "Ventialte Disc", 'Ventilated Disc ': "Ventialte Disc",
                                                   'Ventilated Disc': "Ventialte Disc", 'Solid Disc ':"Solid Disc" ,'Disc ': "Disc",
                                                   'Ventialte Disc': "Ventialte Disc", 'Ventilated Drum': "Ventialte Disc" , 'Drum ': "Drum",
                                                   'Vacuum assisted hydraulic dual circuit w': "Vacuum assisted hydraulic dual circuit w"})

In [30]:
tr = ['4.7 metres', '5.3 metres', '4.9 meters', '5.2 meters', '13.51', '4.6 metres', '5.4 metres', '4.8 metres', '5.1 meters', '5.6 metres', '5.2m',
       '6.15 metres ', '4.4 meters', '6 metres', '5.8 metres', '5.2 metres', '5.61 metres', '5.6', '5.9 m', '5.95 metres', '4.6m', '5.55', '5.20 m',
       '6.2 metres', '6.0 metres', '4.9 metres', '5.1m', '5.5 metres', '5.55 metres', '5.3metres', '4.97 metres', '5.3 meters', '5.05 metres', '5.4',
       '5.65 metres', '5.3 m', '4.5 metres', '4.9', '4.6metres', '4.7 m', '6.4 meters', '5.8', '6.2 Metre', '5.9metres', '5.5 m', '5.7 metres', '4.9 m',
       '5.75 metres', '5.35 metres', '4.8 Meters', '5.0metres', '5.4 m', '5.5m', '4.7m', '5', '5.2', '5.1', '5.05 meters', '5.55 m', '5.25 metres', '4.9m',
       '4.5', '5.0m', '5.9 metres', '5.3', '4.8metres', '4.7 Metre', '5.9 meters', '4.35mm', '4.9metres', '4.1 m', '5.39', '5.4 meters', '5 metres', '5m',
       '4.85metres', '4.5metres', '5.6 meters', '5.1 metres', '6.28 metres', '5.6metres', '5.0 metres', '4 m', '5.1metres', '5.75m', '4.65 metres', '5.1 m',
       '4.0 metres', '5.25 meters', '5.42 m', '5.42 metres', '5.2metres', '5.45 metres', '6.15 metres', '6.05 meters', '4.65metres', '5.15 metres ', '4.7',
       '6.25 meters', '6.45 metres ', '6.1 m', '4.4metres', '5.8 Metre', '6.1m', '6.1 metres', '11.5 metres', '4.85 metres', '5.6 m', '5.4 Meter', '5.05 m', 
       '5.8m', '12 metres', '5.62 m', '4metres', '5.03m', '5.5 meters', '5.85 metres', '4.9 Meters', '5.8 meters', '5.2 m', '5.35 m', '5.75meters', '4.6 m',
       '6.25m', '5.92 metres', '6.42', '5.7', '5.6 Meters', '5.59 metres', '11.6m', '6.45 metres', '13.51 metres', '5.3 Meters', '12.40 m', '4.97 m', 
       '5.26 meters', '5.05', '5.4metres', '5.85 m', '5.3 Meter', '5.4m', '4.97 meters', '6.05 metres', '5.9 metres ', '5.85 meters', '5 meters', '5.8 m',
       '4.58metres', '5.6m', '4.85 metres ', '5.25mm', '5.35', '6250mm', '5.15 metres', '11.6 metres',  '236 mm', '4.6 metres ', '5.9m', '5.65 meters', 
       '5.02', '5.5metres', '4.95m',  '6metres', '5.03 m', '4.65 M', '4.7metres', '5.45 metres ']

# Apply a lambda function to filter the values based on your list
df['Turning Radius'] = df['Turning Radius'].apply(lambda y: y if y in tr else np.nan)
# Extract both integer and decimal parts and convert to float
df['Turning Radius'] = pd.to_numeric(df['Turning Radius'].str.extract('(\d+\.\d+|\d+)')[0], errors='coerce')

In [31]:
ts = ['150 Kmph', '150 kmph', '172km/hr', '190 Kmph', '180 Kmph', '165 Kmph', '172 Kmph', '175 kmph', '195 Kmph', '250 kmph', '236 Kmph', '222 Kmph', 
      '152 kmph', '240 Kmph', '250kmph', '176 Kmph', '170 Kmph', '191km/hr', '232 Kmph', '233 Kmph', '137km/hr', '220 Kmph', '215 Kmph', '154.19 kmph', 
      '183 Kmph', '226 Kmph', '160 kmph', '156 Kmph', '185kmph', '157 Kmph', '165km/hr', '159.4 kmph', '250 Kmph', '135 Kmph', '185 Kmph', '200 Kmph', 
      '230 Kmph', '195.68 kmph', '168 Kmph', '155 Kmph', '177.22 kmph', '163 Kmph', '164 Kmph', '186.08 kmph', '172 kmph', '171.43 kmph', '155 kmph', 
      '178.55 kmph', '182 Kmph', '186km/hr', '189 kmph', '165 Km', '175 Kmph', '227km/hr', '164.26 kmph', '194 Kmph', '242km/hr', '158km/hr', '160 Kmph',
      '190 kmph', '157 kmph', '225 Kmph', '215 kmph', '145 Kmph', '160km/hr', '178 kmph', '164.1km/hr', '158 Kmph', '190km/hr', '196 Kmph', '165 kmph', 
      '171.92 Kmph', '188 Kmph', '228 kmph', '146 Kmph', '202 km/hr', '144km/hr', '213 kmph', '212 Kmph', '250Kmph', '237 kmph', '196km/hr', '208 Kmph',
      '202km/hr', '219 kmph', '217 kmph', '169 kmph', '210 Kmph', '190 Kmph ', '80 kmph', '95 kmph', '207 Kmph', '167.99kmph', '162.9km/hr', '185km/hr',
       '231 kmph', '140 kmph', '161km/hr', '156km/hr', '140 Kmph', '105 kmph', '169 Kmph', '117 Kmph', '179 kmph', '210 kmph', '205 Kmph', '185 kmph', 
       '242 Kmph', '105km/hr', '148km/hr', '186 Kmph', '145kmph', '155.8 kmph', '229 kmph', '156 kmph', '224 Kmph', '149 Kmph', '154km/hr', '242 kmph', 
       '137 kmph', '228km/hr', '164.26kmph', '152Km/Hour', '235 Kmph', '164 kmph', '154 Kmph', '212 kmph', '183km/hr', '144.93 kmph', '235kmph', '243Kmph',
      '205km/hr', '176 kmph', '234 Kmph', '186 kmph', '151.63 kmph', '206 kmph', '240 kmph', '157.3 kmph', '225Kmph', '128.84kmph', '237 Kmph', '230Kmph',
      '202 Kmph', '247 Kmph', ' 185 Kmph', '162 kmph', '151.2 kmph', '200.7 kmph', '117 kmph', '222km/hr', '130 Kmph','189 Kmph', '220 kmph', '175.61 kmph',  
      '163 km/hr', '180km/hr', '138 kmph', '152km/hr', '170km/hr', '230km/hr', '136 kmph',  '151.11 kmph', '151.63 Kmph', '165 kmph ', '188 kmph', '175km/hr',
      '193 kmph', '225km/hr', '217 Kmph', '204 Kmph', '219 Kmph', '235km/hr', '130 kmph', '163.81kmph', '191 Kmph', '221Kmph', '221 Kmph', '222 kmph', 
      '199 Kmph', '228 Kmph', '239 Kmph', '163km/hr', '162.9 kmph', '235 kmph', '234 kmph', '164.5 Kmph', '250km/hr', '228kmph', '183 kmph', '232km/hr',
      '210km/hr', '195km/hr', '206km/hr', '161.7 Kmph', '141km/hr', '210 km/hr', '133.9 kmph', '230 kmph', '233 kmph', '190kmph', '160.58kmph', '120 Kmph',
      '225 kmph', '182km/hr', '181 kmph', '171km/hr', '224 kmph', '195 kmph', '86 Kmph', '180.5 kmph', '161 Kmph', '177km/hr', '150km/hr', '117km/hr', 
      '155km/hr', '163.81 kmph', '192 kmph', ' 160 Kmph', '162km/hr', '126 Kmph', '173 kmph', '151 Kmph', '170.41 kmph', '229 Kmph', '241 Kmph', '310 Kmph',
      '170 kmph', '259 Kmph', '200km/hr', '144.57kmph', '154.19 Kmph', '142.95kmph', '168.56 kmph', '275 Kmph', '144 Kmph', '171 Kmph', '155kmph']

# Apply a lambda function to filter the values based on your list
df['Top Speed'] = df['Top Speed'].apply(lambda y: y if y in ts else np.nan)
# Extract both integer and decimal parts and convert to float
df['Top Speed'] = pd.to_numeric(df['Top Speed'].str.extract('(\d+\.\d+|\d+)')[0], errors='coerce')


In [32]:
acc = ['15.05 Seconds', '14.3 Seconds', '14.2 Seconds', '12 Seconds', '13.3 Seconds', '12.36 seconds', '12.9 Seconds', '13.7 Seconds', '12.5 Seconds', 
 '10 Seconds', '4.8 Seconds', '8 Seconds', '7.9 Seconds', '18.6 Seconds', '6.9 Seconds', '6.1', '9.6 Seconds', '13.2 Seconds', '11.9 Seconds', 
 '12.07 Seconds', '8.4 Seconds', '6.7 Seconds', '17.7 seconds', '7.8 Seconds', '13.25 Seconds', '11 Seconds', '10.5 Seconds', '7.7 Seconds', '15 Seconds',
 '13 Seconds', '8.26 Seconds', '15.6 Seconds', '14.5 seconds', '17.44 Seconds', '9.2 Seconds', '6.6 Seconds', '19 Seconds', '12.3 Seconds', '9.0 Seconds',
 '15.7 Seconds', '8.93 Seconds', '14.8 Seconds', '10.61 Seconds', '16.1 Seconds', '14.5 Seconds', '10.52 Seconds', '12.36 Seconds', '12.51 Seconds',
 '14.9 Seconds', '11.90 Seconds', '16 Seconds', '9.64 Seconds', '12.93 seconds', '15.31 Seconds', '13.5 Seconds', '7.5 Seconds', '13.75 Seconds', '12.1 Seconds',
 '14 Seconds', '8.5 Seconds', '8.6 Seconds', '7.4 Seconds', '12.29 Seconds', '12.8  seconds', '14.4 Seconds', '12.6 Seconds', '10.2 Seconds', '12.2 seconds',
 '9.7 Seconds', '11.41 Seconds', '8.9 Seconds', '13.9 Seconds', '11.5 Seconds', '8.6 seconds', '16.5 seconds', '9.8 Seconds', '7.2 Seconds', '10.62 Seconds',
 '12.79 seconds', '5.7', ' 8.2 Seconds', '8.2 Seconds', '5.8 Seconds', '9 Seconds', '9.1 Seconds', '10.3 seconds', '13.57 Seconds', '7.1 Seconds', '11.64 Seconds', 
 '14.1 Sec', '9.3 Seconds', '11.07 Seconds', '10.82 s', '8.8 Seconds', '11.46 Seconds', '9.5 Seconds', '9.9 Seconds', '14.8 seconds', '14.2', '20 Seconds',
 '13.8 Seconds', '30.3 Seconds', '17.86 seconds', '11.7 seconds', '8.1 Seconds', '11.2 Seconds', '12.5 seconds', '17 Seconds', '15 s', '13.90 Seconds', 
 '16.29 seconds', '8.5 seconds', '9.36 Seconds', '17.6 seconds', '5.7 Seconds', '16.8 Seconds', '20.5 seconds', '9.35 Seconds', '16.2 seconds',
 '7.3 Seconds', '15.5 seconds', '15.8 Seconds', '8.7 Seconds', '15.4 Seconds', '13.07 Seconds', '7.5', '6.8 Seconds', '14.20 Seconds', '15.2 Seconds', 
 '13.6 Seconds', '6.1 Seconds', '6.8', '8.3 Seconds', '10.7 Seconds', '10.42Seconds', '11.8 Seconds', '6.5 Seconds', '12.43 Seconds', '6.2 Seconds', 
 '5.9 Seconds', '11.27 Seconds', '7.5 seconds', '13.21 Seconds', '9.6 sconds', '6.0 Seconds', '12.14 seconds', '7.2sec', '7.6 Seconds', '10.6 Seconds', 
 '11.3 Seconds', '6.3 Seconds', '21 seconds', '14.85 Seconds', '10.31 seconds', '30.3 seconds', '10.9 Seconds', '21.9 Seconds', '8.9 seconds', 
 '6.4 Seconds', '10.60 Seconds', '10.8 Seconds', '15.35 Seconds', '11.6 Seconds', '10.5Seconds', '22.3 seconds', '13.4 seconds', '22.39 seconds',
 '16.18 Seconds', '16.26 Seconds', '21.2 Seconds', '13.5 Second', '12.95 seconds', '10.75 Seconds', '5.6 Seconds', '9.7 seconds', '10.1 Seconds',
'16.26 seconds', '6.11 Seconds', '11.68 Seconds', '11.14 Seconds', '7.4', '7.1 seconds', '5.8 sec', '4.4 Seconds','7.8 seconds', '4.5 Seconds', 
 '14.2 seconds', '10.3 Seconds', '7.0 Seconds', '13.42 Seconds', '6.1 seconds', '4.7 Seconds', '11.2', '11.7 Seconds', '12.85 Seconds', '16.2 Seconds', 
 '17.30 Seconds', '8.8 seconds', '15.9 Seconds', '11.5 seconds', '7.2 seconds', '11.43 Seconds', '18.2 seconds', '21 Seconds', '10.83 s', '17.5 Seconds',
 '11.4 Seconds', '12.8 Seconds', '8.4 seconds', '12.2 Seconds', '11.2 seconds', '11.94 Seconds', '9.1 Second', '11.5 Seconds ', '11.3 seconds', 
 '12.35 Seconds', '13.6', '10.8 seconds', '6.35 Seconds', '17.2 seconds', '16.4 seconds', '11.8 seconds', '16.31 Seconds', '13.5 seconds', '9.6 seconds',
 '9.43 Seconds', '3.8 Seconds', '12.98 s','10.06 Seconds', '4.9 Seconds', '16.4 Seconds']

# Apply a lambda function to filter the values based on your list
df['Acceleration'] = df['Acceleration'].apply(lambda y: y if y in acc else np.nan)

# Extract both integer and decimal parts and convert to float
df['Acceleration'] = pd.to_numeric(df['Acceleration'].str.extract('(\d+\.\d+|\d+)')[0], errors='coerce')


In [33]:
#store tyre type
tt = ['Tubeless, Radial', 'Tubeless Radial', 'Radial ', 'Tubeless ', 'Radial', 'Runflat Tyre', 'Tubeless', 'Tubeless,Radial', 'Tubeless,Radial ',
      'Tubeless,Radials', 'Runflat', 'Radial,Tubeless', 'Tubeless Tyres', 'Radial, Tubeless', 'Tubless, Radial', 'Tubeless, Runflat', 'Run-Flat', 
      'Runflat Tyres', 'Tubeless,Runflat', 'Tubeless Radial Tyres']
df['Tyre Type'] = df['Tyre Type'].apply(lambda y: y if y in tt else np.nan)


#mapping values
df['Tyre Type'] = df['Tyre Type'].map({"Tubeless, Radial": 'Tubeless,Radial', "Tubeless Radial":'Tubeless,Radial',
                                       "Radial ": 'Radial', "Tubeless ": 'Tubeless', "Tubeless,Radial ": 'Tubeless,Radial', 
                                       "Tubeless,Radials": 'Tubeless,Radial', 'Radial,Tubeless': "Tubeless,Radial", 
                                       "Radial, Tubeless": 'Tubeless,Radial', "Tubless, Radial": 'Tubeless,Radial',
                                       "Tubeless, Runflat": 'Tubeless,Runflat', "Run-Flat": 'Runflat', "Runflat Tyres": 'Runflat',
                                       "Tubeless Radial Tyres" : "Tubeless,Radial", "Runflat Tyre": 'Runflat', "Tubeless Tyres": 'Tubeless',
                                       "Radial": 'Radial', "Tubeless": 'Tubeless', "Tubeless,Radial": 'Tubeless,Radial', "Runflat": 'Runflat',
                                       "Tubeless Tyres": 'Tubeless', "Tubeless,Runflat": 'Tubeless,Runflat'
                                       })


In [34]:
#store alloy wheel size
Alloy = ['12', '13', '14', '16', '17', '18', '19', '20', '21', 'R15', 'R16', 'R18', 'R19']
df['Alloy Wheel Size'] = df['Alloy Wheel Size'].apply(lambda x: x if x in Alloy else np.nan)

# Replace 'R' values with corresponding numbers
df['Alloy Wheel Size'] = df['Alloy Wheel Size'].map({'12': "12", '13': "13", '14': "14", '15': "15", '16': "16", '17': "17", '18': "18", '19': "19", 
                                                     '20': "20", '21': "21", 'R15': '15', 'R16': '16', 'R18': '18', 'R19': '19'})


In [35]:
#store the no door numbers
ndn = ['2', '3', '4', '5', '6']
df['No of Doors'] = df['No of Doors'].apply(lambda x: x if x in ndn else np.nan)

In [36]:
df['Cargo Volume'] = pd.to_numeric(df['Cargo Volume'].str.extract('(\d+)')[0], errors='coerce').astype('Int64')

In [37]:
df = df.to_csv("./Car_Dheko_datasets/Cleaned_car_dheko.csv", index=False)

In [38]:
df = pd.read_csv("./Car_Dheko_datasets/Cleaned_car_dheko.csv", low_memory=False)

In [39]:
df.shape

(8369, 189)

In [40]:
import pandas as pd
df = pd.read_csv('./Car_Dheko_datasets/Cleaned_car_dheko.csv', low_memory = False)

In [41]:
df.shape

(8369, 189)

In [42]:
#Feature use
columns_drop = ['it', 'Power Steering', 'Power Windows Front', 'Air Conditioner', 'Heater', 'Fog Lights Rear', 'Fog Lights Front', 'Anti Lock Braking System',
                'Central Locking', 'Remote Fuel Lid Opener', 'Power Windows Rear', 'Air Quality Control', 'Remote Trunk Opener', 'Low Fuel Warning Light',
                'Accessory Power Outlet', 'Vanity Mirror', 'Trunk Light','Rear Seat Headrest', 'Leather Seats', 'Leather Steering Wheel', 'Adjustable Steering',
                'Digital Odometer', 'Tachometer', 'Electronic Multi Tripmeter', 'Fabric Upholstery', 'Adjustable Head Lights', 'Manually Adjustable Exterior Rear View Mirror',
                'Power Adjustable Exterior Rear View Mirror', 'Electric Folding Rear View Mirror', 'Rear Window Wiper', 'Day Night Rear View Mirror','Brake Assist',
                'Power Door Locks', 'Child Safety Locks','Anti Theft Alarm','Driver Air Bag','Passenger Air Bag','Passenger Side Rear View Mirror','Halogen Headlamps',
                'Rear Seat Belts','Seat Belt Warning','Door Ajar Warning', 'Side Impact Beams','Turbo Charger', 'Super Charger','Dvd Player', 'Rear Reading Lamp',
                'Rear Seat Centre Arm Rest','Cup Holders Front','Cup Holders Rear','Multifunction Steering Wheel','Rear ACVents', 'Seat Lumbar Support', 
                'Glove Compartment','Digital Clock','Dual Tone Dashboard','Outside Temperature Display','Height Adjustable Driver Seat', 'Rear Window Washer',
                'Rear Window Defogger','Alloy Wheels','Integrated Antenna','Rear Spoiler','Sun Roof','Outside Rear View Mirror Turn Indicators', 'Front Impact Beams',
                'Adjustable Seats','Keyless Entry','Centrally Mounted Fuel Tank','Engine Immobilizer','Engine Check Warning', 'Crash Sensor','Ebd','Anti Theft Device',
                'Rear Camera','Speed Sensing Auto Door Lock','No Of Airbags','Automatic Head Lamps', 'Isofix Child Seat Mounts','Cd Player','Radio','Speakers Front',
                'Speakers Rear','Integrated2Din Audio','Usb Auxiliary Input', 'Bluetooth','Number Of Speaker','Smart Access Card Entry','Cruise Control',
                'Engine Start Stop Button','Navigation System', 'Chrome Garnish','Roof Rail','Pretensioners And Force Limiter Seatbelts','Hill Assist',
                'Electronic Stability Control', 'Hill Descent Control','Gear Shift Indicator','Driving Experience Control Eco','Smoke Headlamps','Voice Control',
                'Glove Box Cooling', 'Tailgate Ajar','Steering Wheel Gearshift Paddles','Leather Wrap Gear Shift Selector','Ventilated Seats','LEDDRLs',
                'LEDHeadlights','LEDTaillights', 'LEDFog Lamps','Projector Headlamps','Headlamp Washers','Cornering Foglamps', 'Speed Alert',
                'Impact Sensing Auto Door Lock','Android Auto', 'Wireless Phone Charging','Rear Entertainment System', 'Lane Change Indicator',
                'Luggage Hook And Net','Drive Modes','Remote Climate Control', 'Adjustable Headrest','View360Camera','Sos Emergency Assistance',
                'Geo Fence Alert','Blind Spot Monitor','Wifi Connectivity','Power Boot', 'Removable Convertible Top','Compass','Touch Screen',
                'Cornering Headlamps','Hands Free Tailgate','Smart Key Band','Apple Car Play', 'Chrome Grille','Roof Carrier','Moon Roof','Side Stepper',
                'Lane Watch Camera']
df = df.drop(columns = columns_drop, axis=1)



In [43]:
from datetime import datetime
def model_year(year):
    today = datetime.today().year
    return today - year

df["Car_age"] = df["modelYear"].apply(model_year)
df = df.drop('modelYear', axis=1)

In [44]:
output_path =  "./Car_Dheko_datasets/dropped_car_data_set.csv"
df.to_csv(output_path, index=False)

In [45]:
df = pd.read_csv("./Car_Dheko_datasets/dropped_car_data_set.csv", low_memory=False)

In [46]:
df.shape

(8369, 48)

In [47]:
df.columns
print('done')

done
